In [ ]:
import requests
import re
import sys
import time
import pandas as pd
from langchain_chroma import Chroma
from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector
from langchain_ollama import OllamaEmbeddings
from ollama import Client
import json
import subprocess
import time
import ipaddress

from datetime import datetime
import shlex
import math
from typing import Optional, Dict, Any, List, Tuple
import jsonschema
from jsonschema import Draft202012Validator
import ast

In [ ]:
# ONOS Controller Details
ONOS_BASE_URL = "http://10.23.7.63:8182/onos/v1/flows" #use localhost ip if needed and use correct ONOS port number
USERNAME = "onos"
PASSWORD = "rocks"  # Replace with your ONOS credentials
ONOS_API_ROOT = ONOS_BASE_URL.rsplit("/flows", 1)[0]

# Define sudo password
sudo_password = "password" #your localhost password

ip_to_host = {
    "10.0.1.1": "h1onos",
    "10.0.1.2": "h2onos",
    "10.0.1.3": "h3onos",
    "10.0.1.4": "h4onos"
            }

host_to_ip = {
    "h1": "10.0.1.1",
    "h2": "10.0.1.2",
    "h3": "10.0.1.3",
    "h4": "10.0.1.4",
}

In [247]:
switch_id_for_llm_assurance = None
llm_caller_flag = 0

In [ ]:
def validate_onos_schema(json_input):
    """
    Validates if the input conforms to the ONOS flow rule JSON schema.
    Handles JSON strings, Python dictionaries (single quotes), and checks required fields.
    
    Returns:
        (bool, str, dict): (isValid, message, cleaned_json_object)
    """
    data = None
    
    # --- 1. Parsing and Format Correction ---
    if isinstance(json_input, dict):
        data = json_input
    elif isinstance(json_input, str):
        cleaned_input = json_input.strip()
        # Attempt standard JSON load
        try:
            data = json.loads(cleaned_input)
        except json.JSONDecodeError:
            # Fallback: Handle Python-style dicts (single quotes) often output by LLMs
            try:
                data = ast.literal_eval(cleaned_input)
            except (ValueError, SyntaxError):
                return False, "Invalid JSON syntax: Could not parse string.", {}
    else:
        return False, "Input must be a dict or a JSON string.", {}

    # --- 2. Root Structure Check ---
    if "flows" not in data or not isinstance(data["flows"], list):
        return False, "Missing 'flows' list at root.", data

    if len(data["flows"]) == 0:
        return False, "Flows list is empty.", data

    # --- 3. Flow Rule Detail Check ---
    required_flow_keys = ["priority", "deviceId", "isPermanent"]
    
    for index, flow in enumerate(data["flows"]):
        # Check mandatory keys
        for key in required_flow_keys:
            if key not in flow:
                return False, f"Flow #{index} missing required key: '{key}'.", data

        # Check types
        if not isinstance(flow["priority"], int):
            return False, f"Flow #{index}: 'priority' must be an integer.", data
        
        # specific check for deviceId format
        if not isinstance(flow["deviceId"], str) or not flow["deviceId"].startswith("of:"):
            return False, f"Flow #{index}: 'deviceId' must be a string starting with 'of:'.", data

        # --- 4. Treatment Validation (Actions) ---
        # Treatment is optional for Drop rules, but if present must be valid
        if "treatment" in flow:
            treatment = flow["treatment"]
            if "instructions" in treatment:
                if not isinstance(treatment["instructions"], list):
                    return False, f"Flow #{index}: 'treatment.instructions' must be a list.", data
                
                for instr in treatment["instructions"]:
                    if "type" not in instr:
                        return False, f"Flow #{index}: Instruction missing 'type'.", data
                    
                    i_type = instr["type"]
                    if i_type == "OUTPUT" and "port" not in instr:
                        return False, f"Flow #{index}: OUTPUT instruction requires 'port'.", data
                    if i_type == "QUEUE" and "queueId" not in instr:
                        return False, f"Flow #{index}: QUEUE instruction requires 'queueId'.", data
                    if i_type == "L2MODIFICATION" and "subtype" not in instr:
                        return False, f"Flow #{index}: L2MODIFICATION instruction requires 'subtype'.", data

        # --- 5. Selector Validation (Match Criteria) ---
        if "selector" in flow:
            selector = flow["selector"]
            if "criteria" in selector:
                if not isinstance(selector["criteria"], list):
                    return False, f"Flow #{index}: 'selector.criteria' must be a list.", data
                
                for crit in selector["criteria"]:
                    if "type" not in crit:
                        return False, f"Flow #{index}: Criterion missing 'type'.", data
                    
                    c_type = crit["type"]
                    # Basic mapping check
                    if c_type == "ETH_TYPE" and "ethType" not in crit:
                         return False, f"Flow #{index}: ETH_TYPE requires 'ethType'.", data
                    if c_type in ["IPV4_SRC", "IPV4_DST"] and "ip" not in crit:
                         return False, f"Flow #{index}: {c_type} requires 'ip'.", data
                    if c_type in ["TCP_DST", "TCP_SRC", "UDP_DST", "UDP_SRC"]:
                        # Check for port keys (either tcpPort or udpPort)
                        if not any(k in crit for k in ["tcpPort", "udpPort"]):
                             return False, f"Flow #{index}: {c_type} requires a port number.", data

    return True, "Validation Successful", data

In [248]:
def delete_flow_rule_ONOS(device_id, flow_id):
    """
    Delete a specific flow rule from an ONOS device by device_id and flow_id.
    
    Parameters:
        device_id (str): The ONOS switch ID (e.g., "of:0000000000000002").
        flow_id (str or int): The ONOS flow rule ID (as returned from push or get).
    
    Returns:
        bool: True if deleted successfully, False otherwise.
    """
    url = f"{ONOS_BASE_URL}/{device_id}/{flow_id}"
    headers = {
        "Accept": "application/json"
    }

    try:
        response = requests.delete(url, auth=(USERNAME, PASSWORD), headers=headers)
        if response.status_code in [200, 204]:
            print(f"Successfully deleted flow rule from ONOS (Device ID: {device_id}, Flow ID: {flow_id})")
            return True
        else:
            print(f"Failed to delete flow rule. Status Code: {response.status_code}, Response: {response.text}")
            return False
    except Exception as e:
        print(f"Exception occurred while deleting flow rule: {e}")
        return False


def get_flows_for_device_ONOS(device_id):
    """
    Retrieve all flows for a specific ONOS device (switch).
    
    Parameters:
        device_id (str): The ONOS switch ID (e.g., "of:0000000000000002").
    
    Returns:
        list: A list of flow JSON objects for the given switch.
    """
    url = f"{ONOS_BASE_URL}/{device_id}"
    headers = {"Accept": "application/json"}

    try:
        response = requests.get(url, auth=(USERNAME, PASSWORD), headers=headers)
        response.raise_for_status()
        
        data = response.json()
        return data.get("flows", [])

    except requests.exceptions.RequestException as e:
        print(f"Error fetching flows for device {device_id}:", e)
        return []

def extract_switch_id_ONOS(intent: str):
    """
    Extract the switch ID from a natural language intent for ONOS JSON format.
    
    Parameters:
        intent (str): The natural language intent.
    
    Returns:
        str: Extracted switch ID (e.g., 'of:0000000000000001') or None if not found.
    """
    # Mapping of ordinal words to numeric values
    ordinals = {
        "first": 1,
        "second": 2,
        "third": 3,
        "fourth": 4,
        "fifth": 5,
        "sixth": 6,
        "seventh": 7,
        "eighth": 8,
        "ninth": 9,
        "tenth": 10
    }

    # Match patterns like 'switch 1', 'router 2', 'node 3'
    match = re.search(r'\b(?:switch|router|node|device)(?:\s*number)?\s*(\d+)', intent, re.IGNORECASE)
    if match:
        switch_number = int(match.group(1))
        return f"of:{switch_number:016x}"  # Convert to ONOS 16-digit hex format

    # Match ordinal words (e.g., 'fourth switch', 'second router')
    match = re.search(r'\b(?:switch|router|node|device)\s*(\w+)', intent, re.IGNORECASE)
    if match:
        ordinal_word = match.group(1).lower()
        if ordinal_word in ordinals:
            switch_number = ordinals[ordinal_word]
            return f"of:{switch_number:016x}"  # Convert to ONOS 16-digit hex format

    # Match standalone ordinal words (e.g., 'fourth' without 'switch')
    for word, number in ordinals.items():
        if word in intent.lower():
            return f"of:{number:016x}"  # Convert to ONOS 16-digit hex format

    return None

def push_flow_rule(device_id, flow_json):
    """
    Push a flow rule to ONOS and retrieve the flow ID.
    """
    url = f"{ONOS_BASE_URL}"
    HEADERS = { 
        "Content-Type": "application/json",
        "Accept": "application/json"
        }
    
    try:
        response = requests.post(url, headers=HEADERS, auth=(USERNAME, PASSWORD), data=json.dumps(flow_json))
        if response.status_code in [200, 201]:
            response_data = response.json()
            if "flows" in response_data and len(response_data["flows"]) > 0:
                flow_id = response_data["flows"][0].get("flowId")
                print(f"Successfully pushed flow rule to ONOS (Device ID: {device_id}, Flow ID: {flow_id})")
                return flow_id  # Return the flow ID for later verification
            else:
                print("Flow rule pushed but no Flow ID returned by ONOS.")
                return None
        else:
            print(f"Failed to push flow rule to ONOS. Status Code: {response.status_code}, Response: {response.text}")
            return None
    except Exception as e:
        print(f"Exception occurred while pushing flow rule: {e}")
        return None
    
def verify_flow_rule(device_id, flow_id):
    """
    Verify if the flow rule exists in ONOS using the retrieved Flow ID.
    """
    if not flow_id:
        print("\nInvalid Flow ID provided for verification. Check ONOS response after rule posting.\n")
        return False, None

    url = f"{ONOS_BASE_URL}/{device_id}"
    HEADERS = {"Accept": "application/json"}
    
    try:
        response = requests.get(url, headers=HEADERS, auth=(USERNAME, PASSWORD))
        if response.status_code == 200:
            flows = response.json().get("flows", [])
            for flow in flows:
                if flow.get("id") == flow_id:
                    #print(f"Flow rule exists in ONOS (Flow ID: {flow_id})")
                    return True, flow
            print(f"Flow rule NOT found in ONOS (Flow ID: {flow_id})")
            return False, None
        else:
            print(f"Failed to query ONOS. Status Code: {response.status_code}, Response: {response.text}")
            return False, None
    except Exception as e:
        print(f"Exception occurred while verifying flow rule: {e}")
        return False, None

In [249]:
TRANSLATION_PROMPT_ONOS = """Your task is to transform natural language network intents into JSON-formatted network policies compatible with the ONOS SDN controller.

You only reply in JSON, no natural language. The network intents can represent different traffic control behaviors, such as:

1. **Traffic Forwarding, Queue Assignment, and VLAN Rules:** Define rules for forwarding traffic based on IPv4/IPv6 destination, TCP/UDP ports, and optionally assign traffic to specific queues or vlans.
2. **Blocking or Dropping Rule:** Define rules to drop traffic based on specific match criteria (e.g., source IP, destination IP). In ONOS, this is done by omitting the `"treatment"` field.

### **JSON STRUCTUREs FOR ONOS**

1. **Traffic Forwarding, Queue Assignment, and VLAN Rules:**  

```json
{
    "flows": [
        {
            "priority": <integer>,
            "timeout": <integer>, // Default: 0
            "isPermanent": "true",
            "deviceId": "<switch_id>",
            "treatment": {
                "instructions": [
                    {
                        "type": "QUEUE",
                        "queueId": <integer>
                    },
                    {
                        "type": "L2MODIFICATION",
                        "subtype": "VLAN_ID",
                        "vlanId": <integer> // Example: 100 for VLAN tagging
                    },
                    {
                        "type": "OUTPUT",
                        "port": "<integer>"
                    }
                ]
            },
            "selector": {
                "criteria": [
                    {
                        "type": "ETH_TYPE",
                        "ethType": "<string>" // Example: "0x800" for IPv4
                    },
                    {
                        "type": "IPV4_SRC",
                        "ip": "<ip_address/mask>"
                    },
                    {
                        "type": "IPV4_DST",
                        "ip": "<ip_address/mask>"
                    },
                    {
                        "type": "IP_PROTO",
                        "protocol": <integer> // Example: 6 for TCP, 17 for UDP
                    },
                    {
                        "type": "TCP_DST",
                        "tcpPort": <integer>
                    },
                    {
                        "type": "UDP_DST",
                        "udpPort": <integer>
                    },
                    {
                        "type": "IN_PORT",
                        "port": "<integer>"
                    }
                ]
            }
        }
    ]
}


2. **Blocking or Dropping Rule:**

{
    "flows": [
        {
            "priority": <integer>,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "<switch_id>",
            "selector": {
                "criteria": [
                    {
                        "type": "ETH_TYPE",
                        "ethType": "<string>" // Example: "0x800" for IPv4
                    },
                    {
                        "type": "IPV4_SRC",
                        "ip": "<ip_address/mask>"
                    },
                    {
                        "type": "IPV4_DST",
                        "ip": "<ip_address/mask>"
                    }
                ]
            }
        }
    ]
}

Field Descriptions
priority (Mandatory): Priority level (higher numbers indicate higher priority). For Queue, Blocking or firewall rules, assign a priority greater than 40000.
timeout (Mandatory): Timeout in seconds after which the flow is removed (Default: 0).
isPermanent (Mandatory): "true" (always in quotes, per user preference).
deviceId (Mandatory): Switch ID where the rule is installed.
ethType (Mandatory): Ethernet protocol type. Use "0x800" for IPv4, "0x86DD" for IPv6, "0x806" for ARP.
IPV4_DST (Optional): IPv4 address in CIDR notation (e.g., "10.0.0.1/32"). Include only if explicitly mentioned.
IPV4_SRC (Optional): Source IP address (include only if explicitly mentioned).
IP_PROTO (Optional): Transport layer protocol (6 for TCP, 17 for UDP, 1 for ICMP).
TCP_DST (Optional): TCP destination port (e.g., 80 for HTTP).
UDP_DST (Optional): UDP destination port (e.g., 161 for SNMP).
IN_PORT (Optional): Incoming interface port number (use in port-based forwarding).
QUEUE (Optional): Use "QUEUE" with "queueId" to specify a QoS queue (queue ID is an integer, 0 is default).
OUTPUT (Optional): "OUTPUT" with "port" specifies the output port.
VLAN_ID (Optional): Use "L2MODIFICATION" with "subtype": "VLAN_ID" and "vlanId" to set a VLAN tag.

Rules for Translation
Each "priority" must be unique.
Set priority very high (e.g., 40000) for queue and security related rules.
Do not include VLAN-related fields unless explicitly mentioned in the intent.
Do not include optional fields unless explicitly mentioned in the intent.
Ensure valid ONOS-compliant JSON syntax.
Verify JSON structure before responding.
Always respond in valid JSON format only, without comments, explanations, or additional text.
If the intent cannot be mapped, return an empty JSON object {}."""

In [250]:
CONFLICT_PROMPT_ONOS = """You are tasked with determining if two ONOS flow configuration JSONs directly conflict with each other. **You MUST base your decision SOLELY on the JSON content provided.** Do **NOT** infer intent, use semantic reasoning, or make assumptions beyond the given JSON structure.

---

### **Rules for Conflict Detection (STRICT and LITERAL COMPARISON):**

A **direct conflict** exists **ONLY IF** all the following conditions are met:

#### **1. Matching Traffic Characteristics (Exact or Overlapping Match Criteria)**
Both flows must match **overlapping traffic characteristics** for all the specified fields, meaning they could apply to the **exact same packets**. The following fields are checked:

**EtherType (`ETH_TYPE`)**  
   - Must be present in **both** flows and have **identical** values (e.g., `"0x800"` for IPv4, `"0x86DD"` for IPv6).
   - If missing in either flow, there is **no match**.

**Source and Destination IP (`IPV4_SRC` and `IPV4_DST`)**  
   - If both flows **specify** a source or destination, they must overlap (e.g., `"10.0.0.1/32"` and `"10.0.0.0/24"` overlap).
   - If **either** flow **omits** source or destination, it applies to **all** sources or destinations respectively.

**Protocol (`IP_PROTO`)**  
   - Must be present in **both** flows and have **identical** values (e.g., `6` for TCP, `17` for UDP).
   - If missing in either flow, there is **no match**.

**Transport Layer Ports (`TCP_SRC`, `TCP_DST`, `UDP_SRC`, `UDP_DST`)**  
   - Must be **identical** in both flows if present.
   - If one flow includes a port filter and the other does not, there is **no match**.

**Incoming Port (`IN_PORT`)**  
   - If both flows specify `IN_PORT`, they must be identical.
   - If missing in either flow, there is **no match**.

**Wildcard Matching (Implicit Behavior for Missing Fields)**  
   - If a flow **specifies a field** (e.g., `IPV4_SRC`), it applies **only** to that source.
   - If a flow **does not specify a field** (e.g., `IPV4_SRC`), it applies to **all sources**.
   - General vs. specific distinctions are not classified, but missing fields imply generality (i.e., match all). Conflicts are determined based on whether the two match sets could overlap.
---

#### **2. Contradictory Actions**
Flows that **match the same traffic** only conflict if their actions contradict in one of the following ways:

**Different Output Ports (`OUTPUT` instruction)**  
   - If one flow **forwards** traffic to port `X` and another to port `Y`, it is a **conflict**.

**One Flow Drops, One Flow Forwards**  
   - If one flow **omits `"treatment"`** (implying a drop) and another **forwards traffic**, it is a **conflict**.

**Different Queue Assignments (`QUEUE` instruction)**  
   - If the flows **assign different `queueId` values**, it is a **conflict**.

**Note on Additional Match Fields:**  
   - If one flow includes additional match fields, a conflict may still exist if the effective match space overlaps. Extra fields do not automatically prevent a conflict.

---

#### **3. Priority is Irrelevant for Conflict Detection**
- `priority` **does not** determine a conflict.
- Even if one rule **overrides** another, they **do not conflict** unless their actions contradict.

---

### **Input Format**
You will be provided with **two ONOS JSON flow rules** in the following format:

**Flow 1:**
```json
<JSON for Flow 1>

**Flow 2:**
<JSON for Flow 2>

### **Expected Output Format**
Respond strictly in valid JSON format, using the schema below:

{
    "conflict_status": <integer>,
    "conflict_explanation": "<conflict explanation, if any>"
}

Field Descriptions:
   - conflict_status should be 1 if a direct conflict exists, 0 otherwise.
   - conflict_explanation → A brief explanation if a conflict exists, otherwise an empty string "".

NO EXTRA TEXT, COMMENTS, OR EXPLANATIONS OUTSIDE JSON.
DO NOT return 1 unless you are certain of a direct conflict.
Follow strict field-by-field comparison rules—NO inference beyond given data."""

In [251]:
SLICING_PROMPT = """You are tasked with analyzing a natural language intent to determine if it contains a command to create or use a queue/slice in an OpenFlow switch. You should respond in JSON format.

### Rules for Interpretation:
1. **Queue/Slice Detection:**  
   - The intent is considered related to queue/slice if it contains commands such as:
     - "create queue", "create slices", "slice the network", "implement slicing", "slice the flow", "make flowspace slicing", "do slicing", "slice", "implement queue", "do queuing", "assign queue", "assign slice", or any similar phrasing.
   - If the intent does not mention creating or using a queue/slice, set the field `"use_queue"` to `0`.

2. **Switch, Queue, and Port Identification (Data-plane port/interface):**  
   - If the intent specifies a **switch ID** (e.g., "switch 4" or "openflow:4" or "node 4" or "openflow 4"), populate the `"switch_id"` field with its value.  
   - If the intent specifies a **Queue ID or slice ID** (e.g., "queue 4" or "4th queue" or "fourth queue" or "slice 1" or "first slice"), populate the `"queue_id"` field with its value.  
   - If the intent specifies a **port ID** referring to the device interface/output (e.g., "port 2" or "interface 2" or "ethernet 2" or "output node connector 2" or "second port" or "second interface"), populate the `"port_id"` field with its value. If there are multiple instances of "port_id" present, take the one which indicates the **output port or outgoing interface**.
   - If the intent does not specify a switch ID or queue ID or port ID, set the respective field to an empty string (`""`).

3. **Traffic Type and L4 Destination Port Extraction (Protocol/Service):**
   - Detect the **Layer-4 protocol** if mentioned: `"tcp"` or `"udp"`. Populate this in `"traffic_type"`. If not specified, set `"traffic_type"` to `""`.
   - Detect the **destination application port number** (Layer-4 port), if specified as a number (e.g., "port 80", "UDP port 53") or implied via a service reference such as "HTTP (TCP port 80)". Populate this number (as a string) in `"l4_port"`.  
   - Do **not** confuse the L4 port (e.g., 80/53) with the device/interface port (e.g., switch port 2). The former goes to `"l4_port"`, the latter to `"port_id"`.
   - If multiple L4 ports are mentioned, prefer the **destination/service port** used by the traffic selector (e.g., "traffic destined for port 80"). If still ambiguous, choose the first explicit destination/service port mentioned.
   - If the L4 destination port is not specified, set `"l4_port"` to `""`.

4. **Output Format:**  
   - Respond strictly in valid JSON format adhering to the following schema:

```json
{
  "use_queue": <integer>,
  "switch_id": "<string>",
  "queue_id": "<string>",
  "port_id": "<string>",
  "traffic_type": "<string>",
  "l4_port": "<string>"
}

Field Description:
use_queue: 1 if the intent commands to create or use a queue/slice, 0 otherwise.
switch_id: Switch ID if specified in the intent, otherwise "".
queue_id: Queue/Slice ID if specified in the intent, otherwise "".
port_id: Device/interface port if specified (output/outgoing preferred), otherwise "".
traffic_type: "tcp" or "udp" if specified, otherwise "".
l4_port: Destination application port number (e.g., "80", "53") if specified, otherwise "".

No Additional Text:
Do not include any comments, explanations, or outputs outside the JSON format.

Example Inputs and Outputs:

Input Intent:
"Create a queue in switch 4 on port 3 for slicing the flow."
Output:
{
"use_queue": 1,
"switch_id": "switch 4",
"queue_id": "",
"port_id": "port 3",
"traffic_type": "",
"l4_port": ""
}

Input Intent:
"Send all video traffic through queue 0 of openflow:2."
Output:
{
"use_queue": 1,
"switch_id": "openflow 2",
"queue_id": "0",
"port_id": "",
"traffic_type": "",
"l4_port": ""
}

Input Intent:
"Configure switch 5 for traffic management."
Output:
{
"use_queue": 0,
"switch_id": "switch 5",
"queue_id": "",
"port_id": "",
"traffic_type": "",
"l4_port": ""
}

Input Intent:
"Monitor traffic flow on port 1."
Output:
{
"use_queue": 0,
"switch_id": "",
"queue_id": "",
"port_id": "port 1",
"traffic_type": "",
"l4_port": ""
}

Input Intent:
"In switch 3, if the incoming traffic in port 1 is TCP traffic destined for port 80, then pass it via interface 2, assigning it to queue 0 for prioritized handling."
Output:
{
"use_queue": 1,
"switch_id": "switch 3",
"queue_id": "0",
"port_id": "interface 2",
"traffic_type": "tcp",
"l4_port": "80"
}

Input Intent:
"For node 1, route HTTP (TCP port 80) traffic targeting 10.0.0.3/32 through port 2 with traffic assigned to queue 0."
Output:
{
"use_queue": 1,
"switch_id": "node 1",
"queue_id": "0",
"port_id": "port 2",
"traffic_type": "tcp",
"l4_port": "80"
}

Input Intent:
"Forward UDP traffic on port 53 destined for 10.0.0.9 via interface 5 of switch 2, assigning it to queue 3."
Output:
{
"use_queue": 1,
"switch_id": "switch 2",
"queue_id": "3",
"port_id": "interface 5",
"traffic_type": "udp",
"l4_port": "53"
}
"""

In [252]:
my_models_translate_real = [
"codestral",
"command-r",
"huihui_ai/qwq-abliterated"
]

my_models_conflict_real = [
"huihui_ai/qwq-fusion",
"qwq"
]

context_examples = [3, 6]

default_model = "llama2"

ollama_embedding_url = "http://10.23.7.63:11434"
ollama_server_url = "http://10.23.7.63:11435"  

ollama_emb = OllamaEmbeddings(
    model=default_model,
    base_url=ollama_embedding_url,
)

client = Client(host=ollama_server_url , timeout=120)

In [253]:
# Load custom dataset from CSV
custom_dataset = pd.read_csv('ONOS_intent_translation_dataset_for_LLM_Evaluation.csv')

# Ensure proper column names and format
if not {'instruction', 'output'}.issubset(custom_dataset.columns):
    raise ValueError("The dataset must have 'instruction' and 'output' columns.")

# Split into train and test (50/50 split for example)
#trainset, testset = train_test_split(custom_dataset, test_size=0.5, random_state=42, shuffle=True)
trainset = custom_dataset

In [254]:
def append_intent_to_store(
    file_path,
    nl_intent,
    json_flow_rule,
    device_id,
    flow_id,
    intent_type,
    intent_specificity
):
    """
    Appends an intent record to the IntentStore file in JSONL format.
    """
    record = {
        "nl_intent": nl_intent,
        "json_flow_rule": json_flow_rule,
        "device_id": device_id,
        "flow_id": flow_id,
        "intent_type": intent_type,
        "intent_specificity": intent_specificity
    }
    with open(file_path, "a", encoding="utf-8") as f:
        f.write(json.dumps(record) + "\n")

def read_intents_from_store(file_path):
    """
    Yields each intent record from the IntentStore file.
    """
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            intent = json.loads(line)
            yield intent

In [255]:
def run_LLM_conflict(existing_intent_flow_json, new_intent_flow_json):
    system_prompt = CONFLICT_PROMPT_ONOS

    for model in my_models_conflict_real:
        count = 0
        while True:
            count+=1
            try:
                time.sleep(0.1)

                response = client.generate(model=model,
                    options={'temperature': 0.3, 'num_ctx': 8192, 'top_p': 0.5, 'num_predict': 1024, 'num_gpu': 99},
                    stream=False,
                    system=system_prompt,
                    prompt=f"Flow 1:\n{json.dumps(existing_intent_flow_json, indent=2)}\n\nFlow 2:\n{json.dumps(new_intent_flow_json, indent=2)}",
                    format='json'
                )

                output = response['response'].strip()

                response_json = json.loads(output)

                if 'conflict_status' not in response_json:
                    #print("\nWarning: 'conflict_status' key is missing in the response.\n")
                    break
                else:
                    valid_conflict_response = True
                    conflict_status = response_json.get('conflict_status', 0)
                    # Ensure conflict_status is an integer
                    if isinstance(conflict_status, str):
                        conflict_status = int(conflict_status)

                    return valid_conflict_response, conflict_status, response_json['conflict_explanation']             

            except Exception as e:
                print("Exception found: ", e)
                sys.stdout.flush()
                if(count<15):
                    continue
                else:
                    print("\n",model, " failed to produce valid JSON for conflict info after 15 tries. Going to next model\n")
                    break               
    
    return False, None, None

In [256]:
def conflict(device_id, new_intent_flow_json):
    
    existing_flows = get_flows_for_device_ONOS(device_id)
    #existing_flows = manual_flows_by_device.get(device_id, {}).get("flows", [])

    for existing_flow in existing_flows:
        valid_conflict_response, conflict_status, conflict_details = run_LLM_conflict(existing_flow, new_intent_flow_json)

        if (valid_conflict_response == False):
            return 2, None, None   #2 means here that LLM could not generate valid JSON for conflict reporting
        elif (conflict_status == 1):
            return conflict_status, conflict_details, existing_flow
        
    return 0, None, None

In [ ]:
def run_LLM_IBN(intent, device_id):

    for num_examples in context_examples:

        example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
                [{"instruction": trainset.iloc[0]["instruction"], "output": trainset.iloc[0]["output"]}],
                ollama_emb,
                Chroma,
                input_keys=["instruction"],
                k=num_examples,
                vectorstore_kwargs={"fetch_k": min(num_examples, len(trainset))}
                )
        # Clear and add all remaining examples from the trainset
        example_selector.vectorstore.reset_collection()
        
        for _, row in trainset.iterrows():
            example_selector.add_example({
                "instruction": row["instruction"],
                "output": row["output"]
            })

        example_str = ""
        if num_examples > 0:
            examples = example_selector.select_examples({"instruction": intent})
            example_str = "\n\n\n".join(map(lambda x: "Input: " + x["instruction"] + "\n\nOutput: " + x["output"], examples))
            system_prompt = TRANSLATION_PROMPT_ONOS + example_str + "\n\n\n"  

        for model in my_models_translate_real:
            
            #system_prompt = TRANSLATION_PROMPT_ONOS
            count = 0
            actual_output = None

            while True:
                count+=1
                
                try:
                    time.sleep(0.1)
                    if num_examples > 0:
                        examples = example_selector.select_examples({"instruction": intent})
                        example_str = "\n\n\n".join(map(lambda x: "Input: " + x["instruction"] + "\n\nOutput: " + x["output"], examples))
                        system_prompt = TRANSLATION_PROMPT_ONOS + example_str + "\n\n\n"  
                    
                    response = client.generate(model=model,
                        options={'temperature': 0.6, 'num_ctx': 8192, 'top_p': 0.3, 'num_predict': 1024, 'num_gpu': 99},
                        #options={'device': 'cpu'},
                        stream=False,
                        system=system_prompt,
                        prompt=intent,
                        format='json'
                    )
                    actual_output = response['response']
                    #print("\nTranslated by: ", model)

                    is_valid, val_msg, cleaned_json = validate_onos_schema(actual_output)
                    
                    if not is_valid:
                        actual_output = None

                        if count < 15:
                            continue # Retry generation
                        else:
                            print("\n",model, " failed to produce valid JSON for translation info after 15 tries. Going to next model\n")
                
                    break
                
                except Exception as e:
                    print("Exception on Input: ", e)
                    print("\nCheck example_str same or not: \n",example_str)
                    sys.stdout.flush()
                    if(count<15):
                        continue
                    else:
                        print("\n",model, " failed to produce valid JSON for translation info after 15 tries. Going to next model\n")
                        break 
            
            if actual_output:
                try:
                    
                    flow_json = json.loads(actual_output)
                    
                    for flow in flow_json.get("flows", []):  # Iterate over all flows
                        flow["deviceId"] = device_id  # Replace the device ID

                    #print(json.dumps(flow_json))

                    return True, flow_json
                
                except Exception as e:
                    print("Exception found: ", e)
    
    return False, None

In [258]:
def run_LLM_Slice(intent):

    system_prompt = SLICING_PROMPT
    
    for model in my_models_translate_real:     
        try:
            time.sleep(0.1)             
            response = client.generate(model=model,
                options={'temperature': 0.3, 'num_ctx': 8192, 'top_p': 0.5, 'num_predict': 1024, 'num_gpu': 99},
                #options={'device': 'cpu'},
                stream=False,
                system=system_prompt,
                prompt=intent,
                format='json'
            )
            
            output = response['response'].strip()
            response_json = json.loads(output)
            
            #print("\nCheckpoint*******Exiting LLM Slicing detection\n\n******")
            return response_json            
        
        except Exception as e:
            print("Exception found: ", e)
            sys.stdout.flush()
            continue

In [259]:
def classify_onos_flow_rule(flow_rule):
    """
    Classify ONOS flow rule into a type and compute its specificity.
    Works for both wrapped and standalone flow rule formats.
    Returns: (rule_type: str, specificity: float)
    """

    # Support both: wrapped in "flows" list OR flat dict
    if "flows" in flow_rule:
        flows = flow_rule.get("flows", [])
        if not flows:
            return "unknown", 0.0
        rule = flows[0]
    else:
        rule = flow_rule

    treatment = rule.get("treatment")
    selector = rule.get("selector", {})
    criteria = selector.get("criteria", [])

    # --- Rule Type Detection ---
    if treatment is None:
        rule_type = "security"
    else:
        instructions = treatment.get("instructions", [])
        if not instructions:
            rule_type = "security"
        else:
            instr_types = {instr.get("type", "").upper() for instr in instructions}
            if instr_types == {"NOACTION"}:
                rule_type = "security"
            elif "QUEUE" in instr_types:
                rule_type = "qos"
            elif "OUTPUT" in instr_types:
                rule_type = "forwarding"
            else:
                rule_type = "unknown"

    # --- Specificity Computation ---
    specificity = 0.0
    for crit in criteria:
        specificity += 1
        if "ip" in crit:
            ip = crit["ip"]
            try:
                ip_net = ipaddress.ip_network(ip, strict=False)
                specificity += ip_net.prefixlen / 32.0
            except Exception:
                pass

    return rule_type, specificity

def resolve_onos_conflict(rule1, rule2):
    """
    Resolve conflict between two ONOS rules using type > specificity > priority.
    Handles both LLM-generated-style and ONOS-style rule formats.
    Returns: winner_rule, loser_rule
    """
    type_priority = {"security": 3, "qos": 2, "forwarding": 1}

    # Normalize both rules into wrapped format for consistency
    if "flows" not in rule1:
        rule1 = {"flows": [rule1]}
    if "flows" not in rule2:
        rule2 = {"flows": [rule2]}

    type1, spec1 = classify_onos_flow_rule(rule1)
    type2, spec2 = classify_onos_flow_rule(rule2)

    p1 = rule1["flows"][0].get("priority", 0)
    p2 = rule2["flows"][0].get("priority", 0)

    # Type-based resolution
    if type_priority.get(type1, 0) > type_priority.get(type2, 0):
        return rule1, rule2
    elif type_priority.get(type2, 0) > type_priority.get(type1, 0):
        return rule2, rule1

    # Specificity-based resolution
    if spec1 > spec2:
        return rule1, rule2
    elif spec2 > spec1:
        return rule2, rule1

    # Priority-based resolution
    if p1 > p2:
        return rule1, rule2
    elif p2 > p1:
        return rule2, rule1

    # All equal
    return None, None


def adjust_priority_onos(winner_rule: dict, loser_rule: dict, step: int = 10) -> dict:
    """
    Adjusts the priority of the winning ONOS flow rule so it overrides the losing one.
    Supports both wrapped ("flows": [rule]) and flat rule dicts.

    Args:
        winner_rule (dict): The winning ONOS rule (either format).
        loser_rule (dict): The losing ONOS rule (either format).
        step (int): Minimum priority margin. Default is 10.

    Returns:
        dict: Updated winner_rule with modified priority (in-place).
    """

    # Support wrapped and unwrapped formats
    if "flows" in winner_rule:
        winner_flow = winner_rule["flows"][0]
    else:
        winner_flow = winner_rule

    if "flows" in loser_rule:
        loser_flow = loser_rule["flows"][0]
    else:
        loser_flow = loser_rule

    loser_priority = loser_flow.get("priority", 0)
    winner_priority = winner_flow.get("priority", 0)

    new_priority = max(winner_priority, loser_priority + step)
    winner_flow["priority"] = new_priority

    return winner_rule


def extract_inner_flow(rule):
    return rule["flows"][0] if "flows" in rule else rule

In [ ]:
def end_to_end_IBN(intent):

    #current_time = time.time()
    device_id = extract_switch_id_ONOS(intent)
    global switch_id_for_llm_assurance 
    switch_id_for_llm_assurance = device_id

    status, intent_JSON = run_LLM_IBN(intent, device_id)

    if (status == False):
         return False, None, None, None, None

    conflict_status, conflict_details, which_flow_conflict = conflict(device_id, intent_JSON)

    if((conflict_status != 1) and (conflict_status != 0)):
        print("\nCheck Conflict Detection Module, LLM did not produce a valid JSON for conflict detection.\n")
        return False, None, None, None, None

    elif (conflict_status == 1):

        winner, non_winner = resolve_onos_conflict(intent_JSON, which_flow_conflict)

        winner_rule_inner = extract_inner_flow(winner)
        existing_rule_inner = extract_inner_flow(which_flow_conflict)

        if winner is None:
            print("\nConflict resolution resulted in a tie. The new rule and an existing rule has same type, specificity and priority\n")
            print("\nThe New flow rule:\n", intent_JSON, "\nThe existing flow rule: \n", which_flow_conflict)
            print("\nExisting Flow Rule Location: In switch: ",device_id, "\nConflict Details : \n",  conflict_details)
            return False, "Tie", None, None, None
        elif (winner_rule_inner == existing_rule_inner):
            print("\nExisting Flow Rule that Conflicts: \n", winner)
            print("\nThe New Flow Rule Attempted to Install based on the Given Intent: \n", non_winner)
            return False, "existing_rule_win", None, None, None
        else:
            print("Conflict Resolved. Winner Flow Rule: \n", winner)
            print("\nShadowed Flow Rule: \n", non_winner)
            print("\nShadowed Flow Rule Location: In switch: ",device_id, "\nConflict Details : \n",  conflict_details)
            # After resolving conflict and deciding rule1 is the winner:
            updated_flow_json = adjust_priority_onos(winner, non_winner) #argument order important; winner first.
            intent_JSON = updated_flow_json   

    try:
        flow_id = push_flow_rule(device_id, intent_JSON)
    except Exception as e:
            print("Exception found while installing flow rule: ", e)
            sys.stdout.flush()
            return False, None, None, None, None
    try:
        verification_status, operational_flow_rule = verify_flow_rule(device_id, flow_id)
        if(verification_status == True):
            return True, flow_id, device_id, intent_JSON, operational_flow_rule
    except Exception as e:
            print("Exception found while verifying flow rule: ", e)
            sys.stdout.flush()
            return False, None, None, None, None

In [261]:
def extract_switch_id(intent: str):
    """
    Extract the switch ID from a natural language intent.
    
    Parameters:
        intent (str): The natural language intent.
    
    Returns:
        str: Extracted switch ID (e.g., 'openflow:1') or None if not found.
    """
    # Mapping of ordinal words to numeric values
    ordinals = {
        "first": 1,
        "second": 2,
        "third": 3,
        "fourth": 4,
        "fifth": 5,
        "sixth": 6,
        "seventh": 7,
        "eighth": 8,
        "ninth": 9,
        "tenth": 10
    }

    # Match patterns like 'openflow:1'
    match = re.search(r'openflow[:\s](\d+)', intent, re.IGNORECASE)
    if match:
        return f"openflow:{match.group(1)}"

    # Match patterns like 'switch 1', 'router 2', 'node 3'
    match = re.search(r'\b(?:switch|router|node|device)(?:\s*number)?\s*(\d+)', intent, re.IGNORECASE)
    if match:
        return f"openflow:{match.group(1)}"

    # Match ordinal words (e.g., 'fourth switch', 'second router')
    match = re.search(r'\b(?:switch|router|node|device)\s*(\w+)', intent, re.IGNORECASE)
    if match:
        ordinal_word = match.group(1).lower()
        if ordinal_word in ordinals:
            return f"openflow:{ordinals[ordinal_word]}"

    # Match standalone ordinal words (e.g., 'fourth' without 'switch')
    for word, number in ordinals.items():
        if word in intent.lower():
            return f"openflow:{number}"

    return None

def execute_command(command):
    """
    Runs a command with sudo password automation.
    """
    full_command = f"echo {sudo_password} | sudo -S {command}"
    try:
        result = subprocess.run(full_command, shell=True, capture_output=True, text=True)
        if result.returncode == 0:
            return result.stdout.strip()
        else:
            raise Exception(f"Error executing command: {result.stderr.strip()}")
    except Exception as e:
        return str(e)

def get_switch_port_mapping():
    try:
        # Commands to list port and QoS configurations
        list_ports_command = "sudo -S ovs-vsctl list port"
        list_qos_command = "sudo -S ovs-vsctl list qos"
        # Fetch port and QoS data
        ports_output = execute_command(list_ports_command)
        qos_output = execute_command(list_qos_command)

        # Parse QoS data into a dictionary
        qos_mapping = {}
        current_qos = None
        for line in qos_output.splitlines():
            if line.startswith("_uuid"):
                current_qos = line.split(":")[1].strip()
            elif line.startswith("queues") and current_qos:
                qos_mapping[current_qos] = line.split(":")[1].strip()

        # Create a dictionary to store switch-to-port mapping
        switch_port_dict = {}

        # Parse ports data and check for QoS
        current_port = None
        for line in ports_output.splitlines():
            if line.startswith("name"):
                current_port = line.split(":")[1].strip()
            elif line.startswith("qos") and "[]" not in line and current_port:
                qos_uuid = line.split(":")[1].strip()

                # Extract the OpenFlow switch ID and port number
                if "-" in current_port:
                    switch, port = current_port.split("-")
                    switch_id = f"openflow:{switch[1:]}"  # e.g., "s1" -> "openflow:1"
                    port_number = port[3:]  # e.g., "eth2" -> "2"

                    # Add to dictionary
                    if switch_id not in switch_port_dict:
                        switch_port_dict[switch_id] = []
                    switch_port_dict[switch_id].append(port_number)

                current_port = None

        return switch_port_dict

    except Exception as e:
        print(f"Error: {e}")
        return {}
    
def extract_port_number(text: str):
    """
    Extract the Ethernet port number from a natural language text.
    
    Parameters:
        text (str): The input text containing the port reference.
    
    Returns:
        int: Extracted port number or None if not found.
    """
    # Mapping of ordinal words to numeric values
    ordinals = {
        "first": 1,
        "second": 2,
        "third": 3,
        "fourth": 4,
        "fifth": 5,
        "sixth": 6,
        "seventh": 7,
        "eighth": 8,
        "ninth": 9,
        "tenth": 10
    }

    # Match explicit numbers after keywords
    match = re.search(r'\b(?:port|interface|output\s+node\s+connector|ethernet)\s*(\d+)', text, re.IGNORECASE)
    if match:
        return int(match.group(1))

    # Match ordinal words (e.g., 'second port', 'third interface')
    match = re.search(r'\b(?:port|interface|output\s+node\s+connector|ethernet)\s*(\w+)', text, re.IGNORECASE)
    if match:
        ordinal_word = match.group(1).lower()
        if ordinal_word in ordinals:
            return ordinals[ordinal_word]

    # Match standalone ordinal words (e.g., 'second')
    for word, number in ordinals.items():
        if word in text.lower():
            return number

    return None

def ovs_port_exists(switch_name, port_name, sudo_password=sudo_password):
    import subprocess
    cmd = f"sudo -S ovs-vsctl list-ports {switch_name}"
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True, input=sudo_password + "\n")
    ports = result.stdout.split()
    return port_name in ports


def get_switch_name_from_device_id(device_id):
    # device_id is like 'openflow:2'
    idx = int(device_id.split(':')[1])
    return f"s{idx}onos"


def create_two_queue_for_switch(device_id, port, max_rate=10000000, queue_configs=None):
    """
    Creates queues dynamically for a specific switch and port.
    
    Parameters:
        switch (str): The name of the switch in 'openflow:X' format (e.g., 'openflow:4').
        port (int): The port number on the switch (e.g., 2).
        max_rate (int): Maximum rate for the QoS (default is 10000000).
        queue_configs (list): List of tuples specifying min-rate and max-rate for each queue (default is 2 queues).
    """
    if queue_configs is None:
        # Default to 2 queues with these configurations
        queue_configs = [
            (6000000, 6000000),  # Queue 0: min-rate and max-rate
            (4000000, 4000000)   # Queue 1: min-rate and max-rate
        ]

    # Construct the port name from the input
    #port_name = f"{switch.replace('openflow:', 's')}-eth{port}"
    switch_name = get_switch_name_from_device_id(device_id)
    port_name = f"{switch_name}-eth{port}"

    if not ovs_port_exists(switch_name, port_name):
        print(f"Port {port_name} does not exist on bridge {switch_name}!")
        return


    # Construct the QoS command for the specific switch and port
    qos_command = f"sudo -S ovs-vsctl -- set port {port_name} qos=@newqos -- --id=@newqos create qos type=linux-htb other-config:max-rate={max_rate}"
    for i, (min_rate, max_rate) in enumerate(queue_configs):
        qos_command += f" queues:{i}=@q{i}"
    for i, (min_rate, max_rate) in enumerate(queue_configs):
        qos_command += f" -- --id=@q{i} create queue other-config:min-rate={min_rate} other-config:max-rate={max_rate}"

    # Execute the command
    print(f"Running: {qos_command}")
    process = subprocess.Popen(qos_command, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate(input=f"{sudo_password}\n")
    if process.returncode == 0:
        print(f"Success:\n{stdout}")
    else:
        print(f"Error:\n{stderr}")


def create_two_queue_for_switch_handler(slicing_info):

    if 'use_queue' in slicing_info:         
            slicing_status = slicing_info['use_queue']
            slicing_switch_id = slicing_info['switch_id']
            slicing_queue_id = slicing_info['queue_id']
            slicing_port_id = slicing_info['port_id']

            if(slicing_status == 1):
                openflow_id = extract_switch_id(slicing_switch_id)
                switch_port_mapping = get_switch_port_mapping()
                print("\nCheckpoint*******Entering Slice/Queue Management\n\n******")

                port_number = extract_port_number(slicing_port_id)
                
                if openflow_id not in switch_port_mapping :
                    
                        print("\n\nQueue was not installed in ",openflow_id, "\nInstalling now on interface: ", port_number,"\n")
                        print("\nCheckpoint*******Entering queue creation\n\n******")

                        create_two_queue_for_switch(
                            device_id=openflow_id,  port=port_number,
                            queue_configs=[
                                (6000000, 6000000),  # Queue 0
                                (4000000, 4000000)   # Queue 1
                            ]
                            )   
                else:
                    if str(port_number) not in switch_port_mapping[openflow_id]:

                        print("\n\nQueue was not installed in ",openflow_id, " interface: ", port_number, "\nInstalling now...\n")
                        print("\nCheckpoint*******Entering queue creation\n\n******")

                        create_two_queue_for_switch(
                            device_id=openflow_id,  port=port_number,
                            queue_configs=[
                                (6000000, 6000000),  # Queue 0
                                (4000000, 4000000)   # Queue 1
                            ]
                            )
    

In [262]:
def generate_corrective_action_prompt(intent_nl, operational_flow_rule, device_id, ping_count,
                                      candidate_src_ip, candidate_dst_ip, ping_output):
    
    existing_flows = get_flows_for_device_ONOS(device_id)

    prompt_sections = []

    # General instruction
    prompt_sections.append(
        "You are an SDN network assistant. Your task is to recommend a ranked list of corrective actions "
        "to enforce a **security intent** that failed during assurance testing."
    )

    # Intent
    prompt_sections.append(f"1. **Security Intent (in Natural Language)**:\n{intent_nl}")

    # Translated flow rule
    prompt_sections.append("2. **ONOS Flow Rule for the Security Intent**:")
    prompt_sections.append(json.dumps(operational_flow_rule, indent=2))

    # Existing flows
    prompt_sections.append("3. **Existing Flow Rules in the Same Switch**:")
    prompt_sections.append(json.dumps(existing_flows, separators=(",", ":")))

    # Assurance test result
    prompt_sections.append(
        f"""4. **Assurance Test Result**:
        - Ping Source IP: {candidate_src_ip}
        - Ping Destination IP: {candidate_dst_ip}
        - Ping Count: {ping_count}
        - Ping Output:
        {ping_output}"""
    )

    # Instruction and output format
    prompt_sections.append(
        "---\nNow, based on the above data, generate a ranked list of corrective actions "
        "that can help enforce the security intent more effectively. "
        "You must use only the following predefined corrective actions (and use each only if relevant):\n"
        "1. Correct Match Fields\n"
        "2. Increase Priority\n"
        "3. Fix Action Field\n\n"
        "For each corrective action you suggest:\n"
        "- **For 'Correct Match Fields'**: Specify *exactly* which match fields and values should be set in the flow rule's 'selector.criteria' list (e.g., type: 'IPV4_SRC', ip: '10.0.1.1/32'), and which fields (if any) should be removed or changed in the candidate flow rule.\n"
        "- **For 'Increase Priority'**: Indicate which existing rule(s) are overshadowing the candidate rule, their current priority value(s), and the exact priority value the candidate rule should be set to (must be greater than the overshadowing rule(s)).\n"
        "- **For 'Fix Action Field'**: Only include this suggestion if the candidate rule's 'treatment.instructions' field does not contain only type 'NOACTION'.\n\n"
        "Rank the actions according to what you believe to be the root cause, and explain your reasoning in the 'reasoning' field for each suggestion.\n"
        "Return your answer ONLY in the following strict JSON format (do not add extra explanations or commentary):\n"
        "{\n"
        "  \"recommended_actions\": [\n"
        "    {\n"
        "      \"rank\": 1,\n"
        "      \"action\": \"Correct Match Fields\",\n"
        "      \"suggestion\": {\n"
        "        \"set_fields\": {\"IPV4_SRC\": \"10.0.0.1\", \"IPV4_DST\": \"10.0.0.2\"},\n"
        "        \"remove_fields\": [\"TCP_SRC\"],\n"
        "        \"reasoning\": \"The match fields do not match the intended source and destination; correcting them should enforce the intent.\"\n"
        "      }\n"
        "    },\n"
        "    {\n"
        "      \"rank\": 2,\n"
        "      \"action\": \"Increase Priority\",\n"
        "      \"suggestion\": {\n"
        "        \"conflicting_rules\": [ {\"flow_id\": \"123\", \"priority\": 40000} ],\n"
        "        \"recommended_priority\": 40001,\n"
        "        \"reasoning\": \"The candidate rule is overshadowed by a rule with higher priority; increasing priority will resolve this.\"\n"
        "      }\n"
        "    },\n"
        "    ...\n"
        "  ]\n"
        "}\n"
        "Omit any action that is not relevant to the current deviation."
    )

    return "\n\n".join(prompt_sections)


In [263]:
def Run_assurance_LLM (assurance_prompt):
    
    for model in my_models_conflict_real:    
        try:
            time.sleep(0.1)             
            response = client.generate(model=model,
                options={'temperature': 0.3, 'num_ctx': 8192, 'top_p': 0.5, 'num_predict': 1024, 'num_gpu': 99},
                #options={'device': 'cpu'},
                stream=False,
                system="",
                prompt=assurance_prompt,
                format='json'
            )
            
            output = response['response'].strip()
            response_json = json.loads(output)
            return response_json            
        
        except Exception as e:
            print("Exception found at assurance LLM for corrective action generation: ", e)
            sys.stdout.flush()
            continue

In [264]:
def extract_host_and_ip_onos(flow_data):
    """
    Extract source and destination host names and IPs from ONOS flow rule.
    Supports both wrapped (with 'flows') and flat formats.
    
    Returns: (src_host, dst_host, src_ip, dst_ip)
    """
    # Handle both wrapped and flat formats
    flow = flow_data["flows"][0] if "flows" in flow_data else flow_data

    # Default values
    src_ip = dst_ip = None

 # Extract criteria from selector
    criteria = flow.get("selector", {}).get("criteria", [])
    for criterion in criteria:
        if criterion.get("type") == "IPV4_SRC":
            src_ip = criterion["ip"].split("/")[0]
        elif criterion.get("type") == "IPV4_DST":
            dst_ip = criterion["ip"].split("/")[0]

    src_host = ip_to_host.get(src_ip, "Unknown") if src_ip else None
    dst_host = ip_to_host.get(dst_ip, "Unknown") if dst_ip else None

    return src_host, dst_host, src_ip, dst_ip

def execute_command_full(command, timeout):
    """Execute a command and capture full output, waiting for all responses."""
    try:
        process = subprocess.Popen(
            command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )

        output_lines = []
        start_time = time.time()

        while True:
            line = process.stdout.readline()
            if line:
                output_lines.append(line.strip())  # Store the output line
                #print(line.strip())  # Print live output (optional)

            # Check if process has completed
            if process.poll() is not None:
                break

            # Timeout check
            if time.time() - start_time > timeout:
                process.terminate()  # Stop process if timeout occurs
                raise TimeoutError(f"Command timed out after {timeout} seconds")

        # Capture remaining output
        remaining_output, _ = process.communicate()
        if remaining_output:
            output_lines.append(remaining_output.strip())

        return "\n".join(output_lines)

    except Exception as e:
        return str(e)
    
def get_mininet_host_pid(src_host):
    """
    Robustly get the PID of a Mininet host process (e.g., 'h1' or 'h1onos') regardless of user.
    Looks for a process with command containing 'mininet:<src_host>'.
    """
    try:
        ps_output = subprocess.check_output(["ps", "-eo", "pid,args"], text=True).strip().splitlines()
    except subprocess.CalledProcessError as e:
        raise RuntimeError("Failed to run 'ps -eo pid,args'") from e

    for line in ps_output:
        if f"mininet:{src_host}" in line and "grep" not in line:
            parts = line.strip().split(None, 1)
            if len(parts) == 2:
                pid_str, cmd = parts
                try:
                    pid = int(pid_str)
                    #print(f"[DEBUG] Matched host '{src_host}' → PID {pid}")
                    return pid
                except ValueError:
                    continue

    raise RuntimeError(f"No Mininet host process found for '{src_host}'.")

def ONOS_assurance_for_security_intent(src_ip, dst_ip, ping_count=2):
    
    all_ips = ["10.0.1.1", "10.0.1.2", "10.0.1.3", "10.0.1.4"]  # Customize this list as per your topology
    global llm_caller_flag
    llm_caller_flag = 0

    def perform_ping(source_ip, target_ip):

        source_host = ip_to_host.get(source_ip)
        if not source_host:
            print(f"[WARNING] Unknown source host for IP: {source_ip}")
            return None
        try:
            host_pid = get_mininet_host_pid(source_host)
        except Exception as e:
            print(f"[ERROR] Cannot get PID for host {source_host}: {e}")
            return None
        ping_cmd = f"echo {sudo_password} | sudo -S mnexec -a {host_pid} ping -c {ping_count} {target_ip}"
        return execute_command_full(ping_cmd, timeout=15)

    # Case 1: src and dst IP both present
    if src_ip and dst_ip:
        print(f"[INFO] Testing {src_ip} to {dst_ip}")
        output = perform_ping(src_ip, dst_ip)
        if output and "100% packet loss" in output:
            print(f"[PASS] Intent effective. Traffic from {src_ip} to {dst_ip} is blocked.")
        elif output and "0% packet loss" in output:
            print(f"[FAIL] Intent NOT effective. Traffic from {src_ip} to {dst_ip} is NOT blocked.")
            llm_caller_flag = 1
        else:
            print("[WARN] Inconclusive result. Ping output:\n", output)
            llm_caller_flag = 1
        return ping_count, src_ip, dst_ip, output

    # Case 2: src_ip is None → test all src IPs → dst_ip
    elif src_ip is None and dst_ip:
        print(f"[INFO] Testing multiple sources to {dst_ip}")
        for candidate_src in all_ips:
            if candidate_src == dst_ip:
                continue
            output = perform_ping(candidate_src, dst_ip)
            if output and "100% packet loss" in output:
                continue  # this one is blocked, good
            elif output and "0% packet loss" in output:
                print(f"[FAIL] Intent NOT effective. {candidate_src} → {dst_ip} was NOT blocked.")
                llm_caller_flag = 1
                return ping_count, candidate_src, dst_ip, output
            else:
                print(f"[WARN] Inconclusive result from {candidate_src} → {dst_ip}:\n{output}")
                llm_caller_flag = 1
                return ping_count, candidate_src, dst_ip, output
        print(f"[PASS] Intent effective. All sources blocked from reaching {dst_ip}.")
        return ping_count, candidate_src, dst_ip, output

    # Case 3: dst_ip is None → test src_ip → all destinations
    elif dst_ip is None and src_ip:
        print(f"[INFO] Testing {src_ip} for multiple destinations")
        for candidate_dst in all_ips:
            if candidate_dst == src_ip:
                continue
            output = perform_ping(src_ip, candidate_dst)
            if output and "100% packet loss" in output:
                continue  # this one is blocked, good
            elif output and "0% packet loss" in output:
                print(f"[FAIL] Intent NOT effective. {src_ip} → {candidate_dst} was NOT blocked.")
                llm_caller_flag = 1
                return ping_count, src_ip, candidate_dst, output
            else:
                print(f"[WARN] Inconclusive result from {src_ip} → {candidate_dst}:\n{output}")
                llm_caller_flag = 1
                return ping_count, src_ip, candidate_dst, output
        print(f"[PASS] Intent effective. {src_ip} blocked from reaching all destinations.")
        return ping_count, src_ip, candidate_dst, output

    else:
        print("[ERROR] Both source and destination IPs are missing. Cannot evaluate intent.")
        llm_caller_flag = 2

def ONOS_assurance_for_qos_intent(src_host, dst_host, src_ip, dst_ip, ping_count=2):

    print("")

def ONOS_assurance_for_forwarding_intent(src_host, dst_host, src_ip, dst_ip, ping_count=2):

    print("")

In [265]:
def correct_match_fields(candidate_flow, set_fields, remove_fields):
    """
    Update candidate_flow's match fields: set specified, remove specified.
    set_fields: dict, e.g. {'IPV4_SRC': '10.0.2.2/32'}
    remove_fields: list of type strings
    """
    selector = candidate_flow['selector']
    # Remove fields
    selector['criteria'] = [
        crit for crit in selector['criteria'] if crit['type'] not in remove_fields
    ]
    # Set/update fields
    for k, v in set_fields.items():
        # Remove any existing
        selector['criteria'] = [crit for crit in selector['criteria'] if crit['type'] != k]
        # Correct value key based on ONOS spec
        if k in ["IPV4_SRC", "IPV4_DST"]:
            selector['criteria'].append({'type': k, 'ip': v})
        elif k == "ETH_TYPE":
            selector['criteria'].append({'type': k, 'ethType': v})
        else:
            selector['criteria'].append({'type': k, 'value': v})
    candidate_flow['selector'] = selector
    return candidate_flow

def increase_priority(candidate_flow, recommended_priority):
    candidate_flow['priority'] = recommended_priority
    return candidate_flow

def fix_action_field(candidate_flow):
    """
    Make sure treatment.instructions is only [{'type': 'NOACTION'}]
    """
    candidate_flow['treatment'] = {
        "instructions": [{"type": "NOACTION"}]
    }
    return candidate_flow

def parse_and_execute_corrective_actions(candidate_flow, llm_response, device_id, flow_id, src_ip, dst_ip):
    """
    candidate_flow: dict, the ONOS flow rule you want to fix
    existing_flows: list of dicts, flows on the same device
    llm_response: dict, output from LLM with 'recommended_actions'
    intent_drift_fn: function that checks for intent drift, returns True if drift still exists, False if resolved
    push_flow_fn: function to push the modified flow to ONOS
    """
    actions = sorted(llm_response["recommended_actions"], key=lambda x: x["rank"])
    for action_item in actions:
        action = action_item["action"]
        suggestion = action_item["suggestion"]
        print(f"Triggering action: {action} (rank {action_item['rank']})")

        if action == "Correct Match Fields":
            set_fields = suggestion.get("set_fields", {})
            remove_fields = suggestion.get("remove_fields", [])
            candidate_flow = correct_match_fields(candidate_flow, set_fields, remove_fields)
        elif action == "Increase Priority":
            recommended_priority = suggestion.get("recommended_priority")
            candidate_flow = increase_priority(candidate_flow, recommended_priority)
        elif action == "Fix Action Field":
            candidate_flow = fix_action_field(candidate_flow)
        else:
            print(f"Unknown action: {action}")

        # Push the flow to ONOS
        # First remove the old one
        delete_flow_rule_ONOS(device_id, flow_id)

        try:
            flow_id = push_flow_rule(device_id, candidate_flow)
        except Exception as e:
                print("Exception found while installing flow rule: ", e)
                sys.stdout.flush()
        try:
            verification_status, operational_flow_rule = verify_flow_rule(device_id, flow_id)
            if(verification_status == True):
                print("\nCorrected Flow Rules Installed Successfully\n")
        except Exception as e:
                print("Corrected Flow Rules Failed to be Installed Successfully. Exception found while verifying flow rule: ", e)
                sys.stdout.flush()

        global llm_caller_flag

        ONOS_assurance_for_security_intent(src_ip, dst_ip)

        # Check intent drift (assurance) after each action
        
        if llm_caller_flag == 0:
            print(f"Intent deviation resolved after action: {action}")
            return True  # Deviation fixed

        print(f"Deviation not fixed after action: {action}, proceeding to next action...")

    print("\n\nAll suggested corrective actions exhausted, but deviation remains. Escalate the issue to the Operator.\n")
    return False  # Deviation not fixed


In [ ]:
#QoS verification helper functions

_UNITS_BYTES = {"KBytes":1024, "MBytes":1024**2, "GBytes":1024**3, "TBytes":1024**4, "Bytes":1}
_UNITS_BITS  = {"Kbits/sec":1e3, "Mbits/sec":1e6, "Gbits/sec":1e9, "bits/sec":1.0}

def build_pin_selector_forward(src_ip: str, dst_ip: str, dport: int, proto: str = "tcp"):
    proto = proto.lower()
    criteria = [
        {"type": "ETH_TYPE", "ethType": "0x800"},
        {"type": "IP_PROTO", "protocol": 6 if proto == "tcp" else 17},
        {"type": "IPV4_SRC", "ip": f"{src_ip}/32"},
        {"type": "IPV4_DST", "ip": f"{dst_ip}/32"},
    ]
    if proto == "tcp":
        criteria.append({"type": "TCP_DST", "tcpPort": dport})
    else:
        criteria.append({"type": "UDP_DST", "udpPort": dport})
    return {"criteria": criteria}

def build_pin_selector_reverse(src_ip: str, dst_ip: str, sport: int, proto: str = "tcp"):
    # reverse direction (dst->src); for TCP we match TCP_SRC=port, for UDP we match UDP_SRC=port
    proto = proto.lower()
    criteria = [
        {"type": "ETH_TYPE", "ethType": "0x800"},
        {"type": "IP_PROTO", "protocol": 6 if proto == "tcp" else 17},
        {"type": "IPV4_SRC", "ip": f"{dst_ip}/32"},
        {"type": "IPV4_DST", "ip": f"{src_ip}/32"},
    ]
    if proto == "tcp":
        criteria.append({"type": "TCP_SRC", "tcpPort": sport})
    else:
        criteria.append({"type": "UDP_SRC", "udpPort": sport})
    return {"criteria": criteria}

def build_output_only_treatment(out_port: int | str):
    return {"instructions": [{"type": "OUTPUT", "port": str(out_port)}]}

def build_queue_and_output_treatment(queue_id: int, out_port: int | str):
    return {"instructions": [{"type": "QUEUE", "queueId": int(queue_id)}, {"type": "OUTPUT", "port": str(out_port)}]}

def build_pin_flow_body(device_id: str, selector: dict, treatment: dict, priority: int = 65000):
    # ONOS expects an array under "flows"
    return {
        "flows": [{
            "deviceId": device_id,
            "isPermanent": True,
            "priority": priority,
            "selector": selector,
            "treatment": treatment
        }]
    }

# def delete_flow_rule(device_id: str, flow_id: str):
#     url = f"{ONOS_BASE_URL}/{device_id}/{flow_id}"
#     HEADERS = {"Accept": "application/json"}
#     r = requests.delete(url, headers=HEADERS, auth=(USERNAME, PASSWORD), timeout=15)
#     # 204/200/202 = fine; 404 -> already gone is OK
#     if r.status_code not in (200, 202, 204, 404):
#         print(f"[WARN] DELETE {device_id}/{flow_id} failed {r.status_code}: {r.text}")

# ---------- PIN PATH VIA S3 (place below the helpers) ----------
def pin_path_via_s3(src_ip="10.0.1.1", dst_ip="10.0.1.3", dst_port=80, proto="tcp",
                    include_s4_forward_qos=False, s4_queue_id=1):
    """
    Installs high-priority OUTPUT-only pins on s1 and s3 for forward,
    OUTPUT-only pins on s4,s3,s1 for reverse ACK path.
    Optionally installs the s4 forward QoS rule (QUEUE+OUTPUT) if needed.
    Returns dict of {label: (deviceId, flowId)} for cleanup.
    """
    S1, S3, S4 = "of:0000000000000001", "of:0000000000000003", "of:0000000000000004"
    pins = {}

    # Selectors
    sel_fwd = build_pin_selector_forward(src_ip, dst_ip, dport=dst_port, proto=proto)
    sel_rev = build_pin_selector_reverse(src_ip, dst_ip, sport=dst_port, proto=proto)

    # ---- Forward pins (s1->s3->s4). s4 forward QoS rule optional. ----
    # s1 out to s3 (port 2)
    body = build_pin_flow_body(S1, sel_fwd, build_output_only_treatment(2), priority=65000)
    fid = push_flow_rule(S1, body); pins["s1_fwd"] = (S1, fid)

    # s3 out to s4 (port 2)
    body = build_pin_flow_body(S3, sel_fwd, build_output_only_treatment(2), priority=65000)
    fid = push_flow_rule(S3, body); pins["s3_fwd"] = (S3, fid)

    if include_s4_forward_qos:
        # s4 set-queue then out to h3 (port 3)
        body = build_pin_flow_body(S4, sel_fwd, build_queue_and_output_treatment(s4_queue_id, 3), priority=65000)
        fid = push_flow_rule(S4, body); pins["s4_qos_fwd"] = (S4, fid)

    # ---- Reverse (ACK) pins (h3->h1) along s4->s3->s1 ----
    # s4 out to s3 (port 2)
    body = build_pin_flow_body(S4, sel_rev, build_output_only_treatment(2), priority=65000)
    fid = push_flow_rule(S4, body); pins["s4_rev"] = (S4, fid)

    # s3 out to s1 (port 1)
    body = build_pin_flow_body(S3, sel_rev, build_output_only_treatment(1), priority=65000)
    fid = push_flow_rule(S3, body); pins["s3_rev"] = (S3, fid)

    # s1 out to h1 (port 3)
    body = build_pin_flow_body(S1, sel_rev, build_output_only_treatment(3), priority=65000)
    fid = push_flow_rule(S1, body); pins["s1_rev"] = (S1, fid)

    return pins

# def unpin_path(pins: dict):
#     """Remove all pinned rules by device/flow ID (safe if already deleted)."""
#     for label, (dev, fid) in pins.items():
#         if fid:
#             delete_flow_rule(dev, fid)

def run_in_host(host_pid: int, cmd: str, timeout: int = 30, require_sudo: bool = True) -> str:
    """
    Run a command inside a Mininet host namespace using mnexec -a <PID>.
    Always uses sudo here to avoid 'Permission denied' when entering namespaces.
    """
    prefix = f"echo {shlex.quote(sudo_password)} | sudo -S "
    full_cmd = f"{prefix}mnexec -a {host_pid} {cmd}"
    return execute_command_full(full_cmd, timeout=timeout)

def ensure_no_iperf_server(host_pid: int, port: int) -> None:
    """
    Attempt to kill any existing iperf3 server on that port in the host namespace.
    """
    # Try pkill first, then kill by pgrep if needed.
    run_in_host(host_pid, f"pkill -f {shlex.quote(f'iperf3 -s -p {port}')} || true", timeout=5, require_sudo=True)
    run_in_host(host_pid, f"bash -lc \"pgrep -af 'iperf3.*-s.*-p {port}' | awk '{{print $1}}' | xargs -r kill -9\"", timeout=5, require_sudo=True)

def start_iperf_server(host_pid: int, port: int, extra_args: str = "") -> None:
    """
    Start iperf3 server (-s) on port (may require sudo for <1024). Run in background.
    Use --one-off to auto-exit after one test.
    """
    ensure_no_iperf_server(host_pid, port)
    cmd = f"bash -lc 'nohup iperf3 -s -p {int(port)} --one-off {extra_args} >/tmp/iperf3_s_{port}.log 2>&1 & echo $!'"
    out = run_in_host(host_pid, cmd, timeout=5, require_sudo=True)
    # Best-effort small delay for readiness
    time.sleep(0.6)

def stop_iperf_server(host_pid: int, port: int) -> None:
    ensure_no_iperf_server(host_pid, port)

def sudo_sh(cmd, timeout=20):
    return execute_command_full(f"echo {shlex.quote(sudo_password)} | sudo -S bash -lc {shlex.quote(cmd)}", timeout=timeout)

def find_bridge_for_device(device_id: str) -> str:
    js = sudo_sh("ovs-vsctl -f json list Bridge", timeout=10)
    data = json.loads(js)
    dpid_hex = device_id.replace("of:","").lower()
    head = data["headings"]
    for row in data["data"]:
        obj = dict(zip(head, row))
        oc = dict(obj["other_config"][1]) if isinstance(obj.get("other_config"), list) else {}
        oc_dpid = (oc.get("datapath-id") or oc.get("datapath_id") or "").lower().replace(":","")
        if oc_dpid == dpid_hex:
            name = obj["name"]
            return name[-1] if isinstance(name, list) else name
    raise RuntimeError("Bridge not found")

def parse_qos_show(iface: str):
    """
    Parse `ovs-appctl -t ovs-vswitchd qos/show IFACE` that looks like:

      QoS: s2onos-eth2 linux-htb
      max-rate: 10000000

      Default:
        burst: 12512
        min-rate: 6000000
        tx_packets: 1868919
        tx_bytes: 267187192
        tx_errors: 0

      Queue 1:
        burst: 12512
        min-rate: 4000000
        tx_packets: 3429
        tx_bytes: 5078775
        tx_errors: 0

    Returns: {"queues": {"default": {...}, "1": {...}}, "raw": <text>, "_qos": {"max-rate": ...}}
    """
    txt = sudo_sh(f"ovs-appctl -t ovs-vswitchd qos/show {shlex.quote(iface)} 2>&1", timeout=8)

    queues = {}
    current = None
    qos_meta = {}

    for raw in txt.splitlines():
        line = raw.strip()

        # Top-level QoS meta (e.g., max-rate)
        m_qos_max = re.search(r'\bmax-rate:\s*(\d+)', line, re.IGNORECASE)
        if m_qos_max and current is None:
            qos_meta["max-rate"] = int(m_qos_max.group(1))

        # Block headers
        if re.match(r'^(Default)\s*:\s*$', line, re.IGNORECASE):
            current = "default"
            queues.setdefault(current, {})
            continue
        m_q = re.match(r'^(?:Queue|queue)\s+(\d+)\s*:\s*$', line)
        if m_q:
            current = m_q.group(1)
            queues.setdefault(current, {})
            continue

        # Inside a block, pick out key:value lines we care about
        if current:
            for key, pat in [
                ("min-rate",   r'\bmin-rate:\s*(\d+)'),
                ("max-rate",   r'\bmax-rate:\s*(\d+)'),
                ("tx_bytes",   r'\btx_bytes:\s*(\d+)'),
                ("tx_packets", r'\btx_packets:\s*(\d+)'),
            ]:
                m = re.search(pat, line, re.IGNORECASE)
                if m:
                    queues[current][key] = int(m.group(1))

    return {"queues": queues, "raw": txt, "_qos": qos_meta}

# # keep snapshot_queue using the patched parser
def snapshot_queue(iface: str, queue_id: int | str):
    parsed = parse_qos_show(iface)
    q = parsed["queues"].get(str(queue_id)) or parsed["queues"].get("default") or {}
    return {
        "min_rate": q.get("min-rate"),
        "max_rate": q.get("max-rate"),
        "tx_bytes": q.get("tx_bytes"),
        "tx_packets": q.get("tx_packets"),
        "_raw": parsed["raw"][:1200]
    }

# def onos_get(path):
#     url = f"{ONOS_API_ROOT}/{path.lstrip('/')}"
#     r = requests.get(url, auth=(USERNAME, PASSWORD), headers={"Accept":"application/json"}, timeout=15)
#     r.raise_for_status()
#     return r.json()

def onos_get(path):
    r = requests.get(f"{ONOS_API_ROOT}/{path.lstrip('/')}", auth=(USERNAME,PASSWORD), headers={"Accept":"application/json"}, timeout=15)
    r.raise_for_status(); return r.json()

def get_iface_for_port(device_id: str, port_no: int | str) -> str:
    """
    Ask ONOS for port annotations to get iface name (e.g., s2-eth2).
    Handles responses shaped like {"ports":[...]} or just a list.
    Falls back to <bridge>-eth<port> if no annotation is found.
    """
    data = onos_get(f"devices/{device_id}/ports")
    # Normalize to a list of port dicts
    if isinstance(data, dict) and "ports" in data:
        ports_list = data["ports"]
    elif isinstance(data, list):
        ports_list = data
    else:
        print("[DEBUG] Unexpected /devices/.../ports payload:")
        try:
            print(json.dumps(data, indent=2)[:1200])
        except Exception:
            print(str(data)[:1200])
        raise RuntimeError("Unexpected ONOS /ports response shape (see DEBUG).")

    target = int(port_no)
    for p in ports_list:
        # p.get("port") is often a string "2" or an int 2
        try:
            p_no = int(p.get("port"))
        except Exception:
            continue
        if p_no == target:
            ann = p.get("annotations") or {}
            name = ann.get("portName") or ann.get("name") or ann.get("port-name")
            if name:
                return name

    # If annotation missing, fall back to bridge-eth<port>
    print(f"[WARN] No iface annotation for {device_id} port {port_no}; falling back to bridge-eth{port_no}")
    br = find_bridge_for_device(device_id)
    return f"{br}-eth{port_no}"

def get_a_flow_rule_ONOS(device_id, flow_id):
    """
    Fetch a single flow rule from ONOS by device_id and flow_id,
    and display it in JSON format.
    """
    if not device_id or not flow_id:
        print("\nBoth device_id and flow_id are required.\n")
        return None

    # Direct endpoint for a single flow rule
    url = f"{ONOS_BASE_URL}/{device_id}/{flow_id}"
    HEADERS = {"Accept": "application/json"}

    try:
        response = requests.get(url, headers=HEADERS, auth=(USERNAME, PASSWORD))
        if response.status_code == 200:
            flow = response.json()
            #print(json.dumps(flow, indent=4))  # Pretty-print JSON
            return flow
        elif response.status_code == 404:
            print(f"\nFlow rule NOT found (Device: {device_id}, Flow ID: {flow_id})\n")
            return None
        else:
            print(f"\nFailed to query ONOS. Status Code: {response.status_code}, Response: {response.text}\n")
            return None
    except Exception as e:
        print(f"\nException occurred while fetching flow rule: {e}\n")
        return None

def _sudo(cmd, timeout=30):
    return execute_command_full(f"echo {shlex.quote(sudo_password)} | sudo -S bash -lc {shlex.quote(cmd)}",
                                timeout=timeout)

def _ns(pid, cmd, timeout=90):
    return execute_command_full(f"echo {shlex.quote(sudo_password)} | sudo -S mnexec -a {pid} bash -lc {shlex.quote(cmd)}",
                                timeout=timeout)

def _parse_iperf_sender(line: str):
    m = re.search(
        r'\s(\d+(?:\.\d+)?)\s*(KBytes|MBytes|GBytes|TBytes|Bytes)\s+'
        r'(\d+(?:\.\d+)?)\s*(Kbits/sec|Mbits/sec|Gbits/sec|bits/sec)\s+.*sender', line)
    if not m:
        raise RuntimeError("iperf sender parse failed: " + line)
    bytes_val = float(m.group(1)) * _UNITS_BYTES[m.group(2)]
    bps_val   = float(m.group(3)) * _UNITS_BITS[m.group(4)]
    return bps_val/1e6, int(bytes_val)

def _iperf_text_summary(client_pid, dst_ip, dst_port, duration, extra_args=""):
    out = _ns(client_pid, f"iperf3 -c {shlex.quote(dst_ip)} -p {int(dst_port)} -t {int(duration)} -f m {extra_args}", 
              timeout=duration+60)
    # prefer final [SUM] sender line; fallback to last 'sender'
    lines = out.strip().splitlines()
    target = None
    for ln in lines:
        if "[SUM]" in ln and "sender" in ln:
            target = ln
    if not target:
        for ln in reversed(lines):
            if "sender" in ln:
                target = ln; break
    # after failing to find 'sender', fallback to last line with 'sec' and 'bits/sec'
    if not target:
        for ln in reversed(lines):
            if "sec" in ln and "bits/sec" in ln:
                target = ln; break
    if not target:
        raise RuntimeError("Could not find iperf sender summary line.")
    mbps, bytes_sent = _parse_iperf_sender(target)
    return {"sender_mbps": mbps, "bytes_sent": bytes_sent, "sender_line": target}

# ---- ONOS flow counters helper (expects your get_a_flow_rule_ONOS) ----
def _flow_counters(flow_json):
    try:
        f = flow_json["flows"][0]
        return int(f.get("packets", 0)), int(f.get("bytes", 0))
    except Exception:
        return None, None

def verify_qos_flow_with_iperf(
    flow_device_id: str, flow_id: str,
    queue_device_id: str, queue_port_no: int, queue_id: int,
    src_ip: str, dst_ip: str, dst_port: int,
    target_mbps: float,
    duration_sec: int = 8, parallel: int = 8, tcp_mss: int = 1200,
    tolerance_pct: float = 10.0,
    pin_path_flows: list | None = None,   # optional: list of (deviceId, out_port, direction) dicts you push via your push_flow_rule
    protocol: str = "tcp",                # <-- NEW
    udp_bw_mbps: float = 50.0,            # <-- NEW (target send rate for UDP)
    udp_len_bytes: int = 1200,            # <-- NEW (datagram size)
):
    """
    Returns a dict result and prints a human summary.
    """
    # Resolve actors
    iface = get_iface_for_port(queue_device_id, queue_port_no)
    client_pid = get_mininet_host_pid(ip_to_host[src_ip])
    server_pid = get_mininet_host_pid(ip_to_host[dst_ip])

    proto = protocol.lower()
    if proto == "tcp":
        extra_args = f"-P {parallel} -M {tcp_mss}"
    else:
        # For UDP you must specify a target rate with -b (pick higher than your cap so the shaper clamps it)
        extra_args = f"-u -b {udp_bw_mbps}M -l {udp_len_bytes} -P {max(1, parallel)}"

    # Optional: push pinning flows if provided (expects caller to craft correct selectors)
    pushed_ids = []
    if pin_path_flows:
        for item in pin_path_flows:
            dev, outp, direction = item["deviceId"], item["out_port"], item["direction"]
            if direction == "forward":
                sel = build_pin_selector_forward(src_ip, dst_ip, dst_port, proto=proto)
            else:
                sel = build_pin_selector_reverse(src_ip, dst_ip, dst_port, proto=proto)
            flow_body = {
                "flows": [{
                    "priority": 65000,
                    "isPermanent": True,
                    "deviceId": dev,
                    "treatment": {"instructions":[ {"type":"OUTPUT","port": str(outp)} ]},
                    "selector": sel
                }]}
            fid = push_flow_rule(dev, flow_body)
            pushed_ids.append((dev, fid))

    # Best-effort offload guard at egress
    try: _sudo(f"ethtool -K {shlex.quote(iface)} gro off gso off tso off", timeout=6)
    except Exception: pass

    # 1) Snapshots before
    #print("\nSnapshot 1: before\n")
    flow_before = get_a_flow_rule_ONOS(flow_device_id, flow_id) or {}
    f_pkts0, f_bytes0 = _flow_counters(flow_before)
    q0 = snapshot_queue(iface, queue_id)

    # 2) Run iperf
    start_iperf_server(server_pid, dst_port)
    time.sleep(0.6)
    t0 = time.time()
    iptxt = _iperf_text_summary(client_pid, dst_ip, dst_port, duration_sec, extra_args=extra_args)
    t1 = time.time()
    stop_iperf_server(server_pid, dst_port)
    time.sleep(0.8)

    # 3) Snapshots after
    #print("\nSnapshot 1: before\n")
    flow_after = get_a_flow_rule_ONOS(flow_device_id, flow_id) or {}
    f_pkts1, f_bytes1 = _flow_counters(flow_after)
    q1 = snapshot_queue(iface, queue_id)

    # 4) Deltas
    elapsed = max(0.001, t1 - t0)
    q_bytes = None if (q0["tx_bytes"] is None or q1["tx_bytes"] is None) else (q1["tx_bytes"] - q0["tx_bytes"])
    q_mbps  = (q_bytes * 8 / elapsed / 1e6) if q_bytes is not None else None
    f_pkts_delta = (f_pkts1 - f_pkts0) if (f_pkts0 is not None and f_pkts1 is not None) else None
    f_bytes_delta = (f_bytes1 - f_bytes0) if (f_bytes0 is not None and f_bytes1 is not None) else None
    f_mbps = (f_bytes_delta * 8 / elapsed / 1e6) if (f_bytes_delta is not None) else None

    # 5) Decision
    rate_ok   = (q_mbps is not None) and (abs(q_mbps - target_mbps) <= (tolerance_pct/100.0)*target_mbps)
    packets_ok= (f_pkts_delta or 0) > 0
    verdict   = "PASS" if (rate_ok and packets_ok) else "FAIL"

    # Print summary
    print("\n=== QoS FLOW VERIFICATION ===")
    print(f"Flow {flow_id} @ {flow_device_id}  → selector should match {protocol} dst {dst_port} to {dst_ip}")
    print(f"Egress iface={iface} queue={queue_id}  target≈{target_mbps:.3f} Mbps  tol=±{tolerance_pct:.0f}%")
    print(f"iperf sender: {iptxt['sender_mbps']:.3f} Mbps  bytes≈{iptxt['bytes_sent']}")
    print(f"Queue Δbytes={q_bytes} over {elapsed:.3f}s  → queue_measured≈{(q_mbps or 0):.3f} Mbps")
    print(f"Flow Δ: packets={f_pkts_delta} bytes={f_bytes_delta}  (flow_measured≈{(f_mbps or 0):.3f} Mbps)")
    print(f"Queue caps (min/max): {q0['min_rate']} / {q0['max_rate']}  →  {q1['min_rate']} / {q1['max_rate']}")
    print("VERDICT:", verdict)

    return {
        "elapsed_sec": elapsed,
        "iperf_sender_mbps": iptxt["sender_mbps"],
        "queue_measured_mbps": q_mbps,
        "queue_delta_bytes": q_bytes,
        "flow_delta_packets": f_pkts_delta,
        "flow_delta_bytes": f_bytes_delta,
        "flow_measured_mbps": f_mbps,
        "verdict": verdict
    }

def ensure_qos_cap(device_id: str, port_no: int, qid: int, min_bps: int, max_bps: int, port_cap_bps: int = 100_000_000):
    iface = get_iface_for_port(device_id, port_no)
    print(f"[INFO] Setting QoS on {iface}: q{qid} min={min_bps} max={max_bps}, root max-rate={port_cap_bps}bps")
    sudo_sh(f"ovs-vsctl --if-exists clear Port {shlex.quote(iface)} qos")
    cmd = (
        "ovs-vsctl "
        f"-- --id=@q create Queue other-config:min-rate={min_bps} other-config:max-rate={max_bps} "
        f"-- --id=@qos create QoS type=linux-htb other-config:max-rate={port_cap_bps} queues:{qid}=@q "
        f"-- set Port {shlex.quote(iface)} qos=@qos"
    )
    print(sudo_sh(cmd))
    print(sudo_sh(f"ovs-appctl -t ovs-vswitchd qos/show {shlex.quote(iface)}"))

In [ ]:
# --- Diamond topology wiring helpers (drop-in) ---

SW_OF = {
    "1": "of:0000000000000001",  # s1
    "2": "of:0000000000000002",  # s2
    "3": "of:0000000000000003",  # s3
    "4": "of:0000000000000004",  # s4
}

HOSTS = {
    "h1": "10.0.1.1",
    "h2": "10.0.1.2",
    "h3": "10.0.1.3",
    "h4": "10.0.1.4",
}

# Host attachment (edge switch, access port) from your Mininet build
HOST_ATTACH = {
    HOSTS["h1"]: (SW_OF["1"], 3),  # h1 -> s1:3
    HOSTS["h2"]: (SW_OF["1"], 4),  # h2 -> s1:4
    HOSTS["h3"]: (SW_OF["4"], 3),  # h3 -> s4:3
    HOSTS["h4"]: (SW_OF["4"], 4),  # h4 -> s4:4
}

# --- New helpers to satisfy the “missing src/dst” rules ---

def choose_dst_for_port(device_id: str, port_no: int) -> str:
    """
    Pick a destination IP that makes forward traffic EXIT on (device_id, port_no).
    Uses your diamond wiring.
    """
    # Final-hop host ports first
    if device_id == SW_OF["1"] and port_no == 3:  # s1 -> h1
        return HOSTS["h1"]
    if device_id == SW_OF["1"] and port_no == 4:  # s1 -> h2
        return HOSTS["h2"]
    if device_id == SW_OF["4"] and port_no == 3:  # s4 -> h3
        return HOSTS["h3"]
    if device_id == SW_OF["4"] and port_no == 4:  # s4 -> h4
        return HOSTS["h4"]

    # Inter-switch egress: choose a host "behind" the far side so packets must traverse this link
    # s1:1->s2 (right), s1:2->s3 (right) → pick a right-side host (h3 default)
    if device_id == SW_OF["1"] and port_no in (1, 2):
        return HOSTS["h3"]
    # s2:2->s4 (right) → pick right-side host; s2:1->s1 (left) → pick left-side host
    if device_id == SW_OF["2"] and port_no == 2:
        return HOSTS["h3"]
    if device_id == SW_OF["2"] and port_no == 1:
        return HOSTS["h1"]
    # s3:2->s4 (right) → right host; s3:1->s1 (left) → left host
    if device_id == SW_OF["3"] and port_no == 2:
        return HOSTS["h3"]
    if device_id == SW_OF["3"] and port_no == 1:
        return HOSTS["h1"]
    # s4 inter-switch (rare in your tests): port1->s2 (left) pick left host; port2->s3 (left) pick left host
    if device_id == SW_OF["4"] and port_no in (1, 2):
        return HOSTS["h1"]

    # Fallback
    return HOSTS["h3"]

def fill_missing_endpoints(plan: dict, device_id: str, port_no: int) -> tuple[str, str]:
    """
    Apply your policy:
      - If only dst is present → choose src that forces egress at (device,port)
      - If only src is present → choose dst that forces egress at (device,port)
      - If both missing → pick both to force egress at (device,port)
      - Ensure src != dst
    """
    src = plan.get("src_ip")
    dst = plan.get("dst_ip")

    # Normalize empty strings to None
    src = src or None
    dst = dst or None

    if dst is None:
        dst = choose_dst_for_port(device_id, port_no)
    if src is None:
        src = choose_src_for_port(device_id, port_no)

    # If they accidentally collide, flip src to the opposite side
    if src == dst:
        # If dst is on right side, move src to left; else move to right
        try:
            dst_edge, _ = _edge_for_ip(dst)
        except Exception:
            dst_edge = SW_OF["4"]  # assume right if unknown
        src = HOSTS["h1"] if dst_edge in (SW_OF["3"], SW_OF["4"]) else HOSTS["h3"]

    return src, dst

def _strip32(ip: str) -> str:
    return ip.split("/", 1)[0]

def find_edge_for_ip(dst_ip: str):
    """Return (deviceId, portNo) for the host that owns dst_ip.
       Prefer static map; fall back to ONOS /hosts if needed."""
    ip = _strip32(dst_ip)
    if ip in HOST_ATTACH:
        return HOST_ATTACH[ip]
    # Optional fallback via ONOS:
    try:
        js = onos_get(f"hosts?ip={ip}")
        lst = js.get("hosts", js if isinstance(js, list) else [])
        if lst:
            locs = lst[0].get("locations") or lst[0].get("location") or []
            if locs:
                dev = locs[0].get("elementId") or locs[0].get("device") or locs[0].get("element")
                port = int(locs[0].get("port"))
                return dev, port
    except Exception:
        pass
    raise RuntimeError(f"Could not infer edge device/port for dst_ip={dst_ip}")

def infer_enforcement_point(plan):
    """
    Decide where to enforce if device/port not fully specified.
    Preference:
      1) If plan.device_id & plan.port_no given → use those
      2) Else enforce at the final hop to the destination host (works for h1..h4).
    """
    device_id, port_no = plan.get("device_id"), plan.get("port_no")
    if device_id and port_no:
        return device_id, port_no
    return find_edge_for_ip(plan["dst_ip"])

def _edge_for_ip(ip: str):
    ip = ip.split("/", 1)[0]
    if ip in HOST_ATTACH:
        return HOST_ATTACH[ip]
    # Optional: try ONOS if not in static map (kept simple)
    try:
        js = onos_get(f"hosts?ip={ip}")
        lst = js.get("hosts", js if isinstance(js, list) else [])
        if lst:
            loc = (lst[0].get("locations") or lst[0].get("location") or [])[0]
            return loc["elementId"], int(loc["port"])
    except Exception:
        pass
    raise RuntimeError(f"No edge mapping for IP {ip}")

def _is(dev: str, n: str) -> bool:
    """dev == SW_OF[n]?"""
    return dev == SW_OF[n]

def choose_src_for_port(device_id: str, port_no: int) -> str:
    """
    Pick a source IP that will make *forward* traffic egress on (device_id, port_no).
    This aligns the traffic direction with your enforcement point so your queue is actually exercised.
    """
    # s1: final-host ports 3/4 → pick a source on the right (h3)
    if _is(device_id, "1"):
        if port_no in (3, 4):        # s1 -> h1/h2
            return HOSTS["h3"]
        elif port_no in (1, 2):      # s1 -> s2/s3 (inter-switch)
            return HOSTS["h1"]       # left host causes egress at s1
    # s2: port2 to s4 → src on left; port1 to s1 → src on right
    if _is(device_id, "2"):
        return HOSTS["h1"] if port_no == 2 else HOSTS["h3"]
    # s3: port2 to s4 → src on left; port1 to s1 → src on right
    if _is(device_id, "3"):
        return HOSTS["h1"] if port_no == 2 else HOSTS["h3"]
    # s4: final-host ports 3/4 → src on left; inter-switch (1/2) → src on right
    if _is(device_id, "4"):
        if port_no in (3, 4):        # s4 -> h3/h4
            return HOSTS["h1"]
        elif port_no in (1, 2):      # s4 -> s2/s3
            return HOSTS["h3"]
    # Fallback
    return HOSTS["h1"]

def make_pin_path_flows(device_id: str, port_no: int,
                        src_ip: str, dst_ip: str, dst_port: int,
                        protocol: str = "tcp"):
    """
    Build a pinning plan so src_ip -> dst_ip forward traffic *must* traverse (device_id, port_no).

    Returns a list of dicts: [{"deviceId":..., "out_port":..., "direction":"forward|reverse"}, ...]
    - For TCP: includes reverse pins (ACK path) to reduce variability.
    - For UDP: forward-only pins (reverse is optional, skipped here).
    """
    proto = protocol.lower()
    is_udp = (proto == "udp")

    pins = []
    src_edge_dev, src_edge_port = _edge_for_ip(src_ip)
    dst_edge_dev, dst_edge_port = _edge_for_ip(dst_ip)

    def add(dev, outp, direction):
        pins.append({"deviceId": dev, "out_port": int(outp), "direction": direction})

    # ----- cases by enforcement point -----

    # s4 final hop to host (port 3 or 4: h3/h4)
    if _is(device_id, "4") and port_no in (3, 4):
        # Forward: steer from the left toward s4 via s1->s3->s4
        # If source is already on the right (h3/h4), nothing to pin on the way in.
        if _is(src_edge_dev, "1"):
            add(SW_OF["1"], 2, "forward")   # s1 -> s3
            add(SW_OF["3"], 2, "forward")   # s3 -> s4
        elif _is(src_edge_dev, "4"):
            pass  # already on s4 side
        elif _is(src_edge_dev, "2"):
            add(SW_OF["2"], 2, "forward")   # s2 -> s4
        elif _is(src_edge_dev, "3"):
            add(SW_OF["3"], 2, "forward")   # s3 -> s4

        # Reverse (TCP): s4 -> s3 -> s1 -> host(src)
        if not is_udp:
            if _is(src_edge_dev, "1"):
                add(SW_OF["4"], 2, "reverse")   # s4 -> s3
                add(SW_OF["3"], 1, "reverse")   # s3 -> s1
                add(SW_OF["1"], src_edge_port, "reverse")  # s1 -> h1/h2
            elif _is(src_edge_dev, "4"):
                add(SW_OF["4"], src_edge_port, "reverse")  # local s4 -> h3/h4

    # s1 final hop to host (port 3 or 4: h1/h2)
    elif _is(device_id, "1") and port_no in (3, 4):
        # Forward: steer from the right toward s1 via s4->s3->s1
        if _is(src_edge_dev, "4"):
            add(SW_OF["4"], 2, "forward")   # s4 -> s3
            add(SW_OF["3"], 1, "forward")   # s3 -> s1
        elif _is(src_edge_dev, "1"):
            pass  # already on s1 side

        # Reverse (TCP): s1 -> s3 -> s4 -> host(src)
        if not is_udp:
            if _is(src_edge_dev, "4"):
                add(SW_OF["1"], 2, "reverse")   # s1 -> s3
                add(SW_OF["3"], 2, "reverse")   # s3 -> s4
                add(SW_OF["4"], src_edge_port, "reverse")  # s4 -> h3/h4
            elif _is(src_edge_dev, "1"):
                add(SW_OF["1"], src_edge_port, "reverse")  # local s1 -> h1/h2

    # s1 inter-switch egress
    elif _is(device_id, "1") and port_no == 2:
        # Forward: use s1->s3->s4 path
        add(SW_OF["1"], 2, "forward")
        add(SW_OF["3"], 2, "forward")
        # Reverse (TCP): s4 -> s3 -> s1 -> host(src)
        if not is_udp:
            add(SW_OF["4"], 2, "reverse")
            add(SW_OF["3"], 1, "reverse")
            add(SW_OF["1"], src_edge_port, "reverse")
    elif _is(device_id, "1") and port_no == 1:
        # Forward: use s1->s2->s4 path
        add(SW_OF["1"], 1, "forward")
        add(SW_OF["2"], 2, "forward")
        # Reverse (TCP): s4 -> s2 -> s1 -> host(src)
        if not is_udp:
            add(SW_OF["4"], 1, "reverse")
            add(SW_OF["2"], 1, "reverse")
            add(SW_OF["1"], src_edge_port, "reverse")

    # s2 inter-switch egress
    elif _is(device_id, "2") and port_no == 2:
        # Forward: s1 -> s2 -> s4
        add(SW_OF["1"], 1, "forward")
        add(SW_OF["2"], 2, "forward")
        if not is_udp:
            add(SW_OF["4"], 1, "reverse")
            add(SW_OF["2"], 1, "reverse")
            add(SW_OF["1"], src_edge_port, "reverse")
    elif _is(device_id, "2") and port_no == 1:
        # Forward: s4 -> s2 -> s1
        add(SW_OF["4"], 1, "forward")
        add(SW_OF["2"], 1, "forward")
        if not is_udp:
            add(SW_OF["1"], 1, "reverse")
            add(SW_OF["2"], 2, "reverse")
            add(SW_OF["4"], src_edge_port, "reverse")

    # s3 inter-switch egress
    elif _is(device_id, "3") and port_no == 2:
        # Forward: s1 -> s3 -> s4
        add(SW_OF["1"], 2, "forward")
        add(SW_OF["3"], 2, "forward")
        if not is_udp:
            add(SW_OF["4"], 2, "reverse")
            add(SW_OF["3"], 1, "reverse")
            add(SW_OF["1"], src_edge_port, "reverse")
    elif _is(device_id, "3") and port_no == 1:
        # Forward: s4 -> s3 -> s1
        add(SW_OF["4"], 2, "forward")
        add(SW_OF["3"], 1, "forward")
        if not is_udp:
            add(SW_OF["1"], 2, "reverse")
            add(SW_OF["3"], 2, "reverse")
            add(SW_OF["4"], src_edge_port, "reverse")

    # s4 inter-switch egress (rare in your tests but supported)
    elif _is(device_id, "4") and port_no in (1, 2):
        # To use s4->(2?3?) link in *forward* direction, src should be on the right.
        # Forward: none if src is already on s4; else steer toward s4 via right path
        if _is(src_edge_dev, "1"):
            add(SW_OF["1"], 2, "forward")  # prefer via s3
            add(SW_OF["3"], 2, "forward")
        # Reverse (TCP): symmetric back to src
        if not is_udp:
            if port_no == 1:
                add(SW_OF["4"], 1, "reverse"); add(SW_OF["2"], 1, "reverse")
                add(SW_OF["1"], src_edge_port, "reverse")
            else:
                add(SW_OF["4"], 2, "reverse"); add(SW_OF["3"], 1, "reverse")
                add(SW_OF["1"], src_edge_port, "reverse")

    else:
        # Fallback: route via s1->s3->s4 with TCP reverse symmetry
        add(SW_OF["1"], 2, "forward")
        add(SW_OF["3"], 2, "forward")
        if not is_udp:
            add(SW_OF["4"], 2, "reverse")
            add(SW_OF["3"], 1, "reverse")
            add(SW_OF["1"], src_edge_port, "reverse")

    # For UDP, we intentionally omit reverse pins (ACK-less).
    # if is_udp:
    #     pins = [p for p in pins if p["direction"] == "forward"]

    # --- Never overshadow the QoS rule on the enforcement port ---
    # Drop any FORWARD pin that sits on the enforcement device+port.
    pins = [p for p in pins
            if not (p["direction"] == "forward"
                    and p["deviceId"] == device_id
                    and int(p["out_port"]) == int(port_no))]

    # For UDP, keep forward-only (already done above)
    if is_udp:
        pins = [p for p in pins if p["direction"] == "forward"]

    # De-duplicate [(dev,port,direction)] triples, keep first
    seen, uniq = set(), []
    for p in pins:
        key = (p["deviceId"], int(p["out_port"]), p["direction"])
        if key in seen: 
            continue
        seen.add(key); uniq.append(p)
    return uniq
    #return pins

def normalize_device_id(dev: str | None) -> str | None:
    if not dev:
        return dev
    dev = dev.strip()

    # openflow:<decimal>
    m = re.fullmatch(r'openflow:(\d+)', dev, flags=re.I)
    if m:
        return f"of:{int(m.group(1)):016x}"

    # openflow:<hex> or openflow:0x<hex>
    m = re.fullmatch(r'openflow:(?:0x)?([0-9a-fA-F]+)', dev, flags=re.I)
    if m:
        return f"of:{int(m.group(1), 16):016x}"

    # of:<decimal>
    m = re.fullmatch(r'of:(\d+)', dev, flags=re.I)
    if m:
        return f"of:{int(m.group(1)):016x}"

    # of:<hex already>
    if dev.lower().startswith("of:"):
        return "of:" + dev[3:].lower()

    return dev

def parse_intent_text(slicing_info, src_ip, dst_ip):

                slicing_queue_id = slicing_info['queue_id'] if (slicing_info['queue_id']) != "" else 1
                proto = slicing_info['traffic_type'] if "udp" in slicing_info['traffic_type'] else ("tcp" if "tcp" in slicing_info['traffic_type'] or "http" in slicing_info['traffic_type'] else "tcp")
                slicing_l4_port = slicing_info['l4_port'] if slicing_info['l4_port'] != "" else 80
                port_no = extract_port_number(slicing_info['port_id'])
                device_id = extract_switch_id(slicing_info['switch_id'])
                device_id = normalize_device_id (device_id)

                return {
                    "protocol": proto,
                    "dst_port": slicing_l4_port,
                    "device_id": device_id,   # may be None
                    "port_no": port_no,       # may be None, it means output interface number
                    "queue_id": slicing_queue_id,
                    "src_ip": src_ip,         # may be None
                    "dst_ip": dst_ip,         # may be None
                }

def intent_to_verifier_args(slicing_info, old_src_ip, old_dst_ip, target_mbps: float = 4.0):
    
    plan = parse_intent_text(slicing_info, old_src_ip, old_dst_ip)

    # 1) Decide enforcement point:
    #    - If device/port specified → use them
    #    - Else → final hop to dst (and if dst missing, we’ll still pick a consistent dst next)
    device_id, port_no = infer_enforcement_point(plan)
    plan["device_id"], plan["port_no"] = device_id, port_no

    # 2) Fill missing src/dst according to your policy (and ensure src != dst)
    src_ip, dst_ip = fill_missing_endpoints(plan, device_id, port_no)
    plan["src_ip"], plan["dst_ip"] = src_ip, dst_ip

    # 3) Build verifier args
    args = {
        "flow_device_id":  device_id,
        "queue_device_id": device_id,
        "queue_port_no":   port_no,
        "queue_id":        plan["queue_id"],
        "src_ip":          src_ip,
        "dst_ip":          dst_ip,
        "dst_port":    plan["dst_port"],   # if your verifier expects dst_port, duplicate the key:
        "dst_port":        plan["dst_port"],
        "target_mbps":     target_mbps,
    }

    # 4) Pin plan (TCP includes reverse, UDP forward-only)
    pins = make_pin_path_flows(device_id, port_no, src_ip, dst_ip, plan["dst_port"], plan["protocol"])
    return args, pins, plan

def delete_flow_rule(device_id: str, flow_id: str):
    url = f"{ONOS_BASE_URL}/{device_id}/{flow_id}"
    r = requests.delete(url, auth=(USERNAME, PASSWORD), headers={"Accept":"application/json"}, timeout=15)
    if r.status_code not in (200,202,204):
        print(f"[WARN] Delete failed for {device_id}/{flow_id}: {r.status_code} {r.text}")

def _pin_selector(direction: str, protocol: str, src_ip: str, dst_ip: str, dst_port: int):
    """Build protocol-aware 5-tuple for forward/reverse."""
    is_udp = protocol.lower() == "udp"
    crit = [
        {"type":"ETH_TYPE","ethType":"0x800"},
        {"type":"IP_PROTO","protocol": 17 if is_udp else 6},
    ]
    if direction == "forward":
        crit += [
            {"type":"IPV4_SRC","ip": f"{src_ip}/32"},
            {"type":"IPV4_DST","ip": f"{dst_ip}/32"},
            ({"type":"UDP_DST","udpPort": dst_port} if is_udp else {"type":"TCP_DST","tcpPort": dst_port}),
        ]
    else:  # reverse
        crit += [
            {"type":"IPV4_SRC","ip": f"{dst_ip}/32"},
            {"type":"IPV4_DST","ip": f"{src_ip}/32"},
            ({"type":"UDP_SRC","udpPort": dst_port} if is_udp else {"type":"TCP_SRC","tcpPort": dst_port}),
        ]
    return {"criteria": crit}

def install_pins_from_plan(pins: list, src_ip: str, dst_ip: str, dst_port: int, protocol: str) -> list:
    """Install the pin flows returned by make_pin_path_flows(...).
       Returns [(deviceId, flowId), ...] for cleanup."""
    pushed = []
    for p in pins:
        dev = p["deviceId"]; outp = str(p["out_port"]); direction = p["direction"]
        flow_body = {
            "flows": [{
                "deviceId": dev,
                "isPermanent": True,
                "priority": 65000,
                "selector": _pin_selector(direction, protocol, src_ip, dst_ip, dst_port),
                "treatment": {"instructions": [ {"type":"OUTPUT","port": outp} ]}
            }]}
        fid = push_flow_rule(dev, flow_body)
        if fid:
            pushed.append((dev, fid))
    return pushed

def unpin_path(pushed: list):
    for dev, fid in pushed:
        delete_flow_rule(dev, fid)

In [ ]:
intent = "Forward TCP traffic on port 80 destined for 10.0.1.3 via interface 2, assigning it to queue 1 for prioritized handling in switch 2."
#intent = "In switch 4, block all IPv4 traffic from 10.0.1.1 to 10.0.1.4 with a high priority, ensuring the switch operates as a firewall."
#intent = "In switch 4, traffic destined for 10.0.1.4 should use port 4."

current_time = time.time()

slicing_info = run_LLM_Slice(intent)

#create_two_queue_for_switch_handler(slicing_info)

deployment_status, flow_id, device_id, translated_flow_rule, operational_flow_rule = end_to_end_IBN(intent)

if 'use_queue' in slicing_info and slicing_info['use_queue'] == 1:
    #Creating one queue
    expected_queue_rate_mbps = 4.0
    port_max = 100_000_000  # 100 Mbps cap for the port QoS root
    min_rate = max_rate = int(expected_queue_rate_mbps * 1_000_000)
    slicing_queue_id = slicing_info['queue_id'] if (slicing_info['queue_id']) != "" else 1
    proto = slicing_info['traffic_type'] if "udp" in slicing_info['traffic_type'] else ("tcp" if "tcp" in slicing_info['traffic_type'] or "http" in slicing_info['traffic_type'] else "tcp")
    slicing_l4_port = slicing_info['l4_port'] if slicing_info['l4_port'] != "" else 80
    port_no = extract_port_number(slicing_info['port_id'])
    
    ensure_qos_cap(device_id, port_no, slicing_queue_id, min_bps=min_rate, max_bps=max_rate, port_cap_bps=100_000_000)

global llm_caller_flag

if (deployment_status == True):
        proc_time_s = (time.time() - current_time)
        print("\n\nSuccessfully translated and installed the rule in ODL SDN Controller. Time taken: ", proc_time_s)
        src_host, dst_host, src_ip, dst_ip = extract_host_and_ip_onos(translated_flow_rule)
        flow_rule_type, flow_rule_specificity = classify_onos_flow_rule(translated_flow_rule)
        
        # Example of appending an intent to IntentStore
        append_intent_to_store(
            "IntentStore_ONOS.jsonl",
            nl_intent=intent,
            json_flow_rule=translated_flow_rule,
            device_id=device_id,
            flow_id=flow_id,
            intent_type=flow_rule_type,
            intent_specificity=flow_rule_specificity
            )
        
        if (flow_rule_type== "security"):
            ping_count, candidate_src_ip, candidate_dst_ip, ping_output = ONOS_assurance_for_security_intent(src_ip, dst_ip)
            if (llm_caller_flag == 1):
                print("\nAsking LLM to generate corrective actions...")
                assurance_LLM_prompt = generate_corrective_action_prompt(intent, operational_flow_rule, device_id, ping_count,
                                    candidate_src_ip, candidate_dst_ip, ping_output)
                llm_response = Run_assurance_LLM (assurance_LLM_prompt)
                print(llm_response)
                parse_and_execute_corrective_actions(operational_flow_rule, llm_response, device_id, flow_id, src_ip, dst_ip)
                 
        elif (flow_rule_type== "qos"):

            target_mbps = 1

            # 1) Resolve intent → concrete args + pin plan
            args, pins_plan, plan = intent_to_verifier_args(slicing_info, src_ip, dst_ip, target_mbps)
            protocol = plan["protocol"]             # "tcp" or "udp"
            dst_port = args["dst_port"] if "dst_tcp_port" in args else args.get("dst_port", 80)

            # 2) Ensure we have a flow_id (find an existing QoS flow on that device/port/queue/L4)

            if not flow_id:
                print("[WARN] Could not find an existing QoS flow matching the intent on the enforcement device.")
                print("If we want this caller to auto-push the QoS flow, we can add a tiny helper for that.")
                # We can still run the verifier; flow deltas will be None but queue counters are sufficient.
            else:
                args["flow_id"] = flow_id

            # 3) Install pin flows so the traffic must traverse the enforcement port
            pushed_pins = install_pins_from_plan(
                pins=pins_plan,
                src_ip=args["src_ip"], dst_ip=args["dst_ip"],
                dst_port=dst_port, protocol=protocol
            )

            try:
                # 4) Call the verifier
                if protocol == "udp":
                    # UDP: send at high rate so shaping is visible (edit the args if your verifier signature differs)
                    verify_qos_flow_with_iperf(
                        flow_device_id=args["flow_device_id"],
                        flow_id=args.get("flow_id"),
                        queue_device_id=args["queue_device_id"],
                        queue_port_no=args["queue_port_no"],
                        queue_id=args["queue_id"],
                        src_ip=args["src_ip"],
                        dst_ip=args["dst_ip"],
                        dst_port=dst_port,          # reuse field name; your function may still call it dst_tcp_port internally
                        target_mbps=args["target_mbps"],
                        duration_sec=8,
                        parallel=1,
                        tolerance_pct=10.0,
                        protocol="udp",
                        udp_bw_mbps=50.0,
                        udp_len_bytes=1200
                    )
                else:
                    # TCP
                    verify_qos_flow_with_iperf(
                        flow_device_id=args["flow_device_id"],
                        flow_id=args.get("flow_id"),
                        queue_device_id=args["queue_device_id"],
                        queue_port_no=args["queue_port_no"],
                        queue_id=args["queue_id"],
                        src_ip=args["src_ip"],
                        dst_ip=args["dst_ip"],
                        dst_port=dst_port,          # if your function uses dst_tcp_port, rename this argument accordingly
                        target_mbps=args["target_mbps"],
                        duration_sec=8,
                        parallel=8,
                        tcp_mss=1200,
                        tolerance_pct=10.0,
                        protocol="tcp"
                    )
            finally:
                # 5) Always clean up pin flows after the test
                unpin_path(pushed_pins)       
             
        elif (flow_rule_type== "forwarding"):
             ONOS_assurance_for_forwarding_intent(src_host, dst_host, src_ip, dst_ip)

        elapsed_time = (time.time() - current_time)
        print("\nTime taken for end-to-end IBN: ",  round(elapsed_time,2))

elif (translated_flow_rule == "Tie"):
    print("\n\nReport to the operator about this conflict resolution issue. Need adjustment to conflict resolution policy.\n")

elif (translated_flow_rule == "existing_rule_win"):
    print("\n\nThe new intent conflicts with an existing one having a higher priority according to current policy, hence the new intent was not installed. See the existing flow rule that conflicts.\n")

else:
    print("\n\nLLM failed to produce meaningful response. Either update context example or model.\n")